In [1]:
# import packages
import pandas as pd
import numpy as np
from datetime import datetime
from random import randint
import copy

In [2]:
codes = {'Arizona Cardinals': 'ARI',
         'Atlanta Falcons': 'ATL',
         'Baltimore Ravens': 'BAL',
         'Buffalo Bills': 'BUF',
         'Carolina Panthers': 'CAR',
         'Chicago Bears': 'CHI',
         'Cincinnati Bengals': 'CIN',
         'Cleveland Browns': 'CLE',
         'Dallas Cowboys': 'DAL',
         'Denver Broncos': 'DEN',
         'Detroit Lions': 'DET',
         'Green Bay Packers': 'GNB',
         'Houston Texans': 'HOU',
         'Indianapolis Colts': 'IND',
         'Jacksonville Jaguars': 'JAX',
         'Kansas City Chiefs': 'KAN',
         'Las Vegas Raiders': 'LVR',
         'Los Angeles Chargers': 'LAC',
         'Los Angeles Rams': 'LAR',
         'Miami Dolphins': 'MIA',
         'Minnesota Vikings': 'MIN',
         'New England Patriots': 'NWE',
         'New Orleans Saints': 'NOR',
         'New York Giants': 'NYG',
         'New York Jets': 'NYJ',
         'Oakland Raiders': 'OAK',
         'Philadelphia Eagles': 'PHI',
         'Pittsburgh Steelers': 'PIT',
         'San Diego Chargers': 'SDG',
         'San Francisco 49ers': 'SFO',
         'Seattle Seahawks': 'SEA',
         'St. Louis Rams': 'STL',
         'Tampa Bay Buccaneers': 'TAM',
         'Tennessee Titans': 'TEN',
         'Washington Football Team': 'WAS',
         'Washington Redskins': 'WAS'
}

divisions = {'ARI': 'NFC West',
         'ATL': 'NFC South',
         'BAL': 'AFC North',
         'BUF': 'AFC East',
         'CAR': 'NFC South',
         'CHI': 'NFC North',
         'CIN': 'AFC North',
         'CLE': 'AFC North',
         'DAL': 'NFC East',
         'DEN': 'AFC West',
         'DET': 'NFC North',
         'GNB': 'NFC North',
         'HOU': 'AFC South',
         'IND': 'AFC South',
         'JAX': 'AFC South',
         'KAN': 'AFC West',
         'LVR': 'AFC West',
         'LAC': 'AFC West',
         'LAR': 'NFC West',
         'MIA': 'AFC East',
         'MIN': 'NFC North',
         'NWE': 'AFC East',
         'NOR': 'NFC South',
         'NYG': 'NFC East',
         'NYJ': 'AFC East',
         'OAK': 'AFC West',
         'PHI': 'NFC East',
         'PIT': 'AFC North',
         'SDG': 'AFC West',
         'SFO': 'NFC West',
         'SEA': 'NFC West',
         'STL': 'NFC West',
         'TAM': 'NFC South',
         'TEN': 'AFC South',
         'WAS': 'NFC East',
         'WAS': 'NFC East'
}

indoor_stadiums = ['State Farm Stadium',
                   'Ford Field',
                   'Edward Jones Dome',
                   'NRG Stadium',
                   'AT&T Stadium',
                   'Georgia Dome',
                   'RCA Dome',
                   'Hubert H. Humphrey Metrodome',
                   'Caesars Superdome',
                   'Lucas Oil Stadium',
                   'Rogers Centre',
                   'U.S. Bank Stadium',
                   'Mercedes-Benz Stadium',
                   'SoFi Stadium',
                   'Allegiant Stadium']

In [3]:
# import game data csv file
game_data = pd.read_csv('nfl-game-data.csv', sep=',', encoding='unicode_escape')

# import team matchups csv file
matchup_data = pd.read_csv('nfl-team-matchup-data.csv', sep=',', encoding='unicode_escape')

In [4]:
# change all team codes to current codes in data
game_data = game_data.replace(['STL', 'OAK', 'SDG'], ['LAR', 'LVR', 'LAC'])
matchup_data = matchup_data.replace(['STL', 'OAK', 'SDG'], ['LAR', 'LVR', 'LAC'])

In [5]:
# change all renamed stadiums to current codes in data
game_data = game_data.replace(['Giants Stadium','Alltel Stadium','University of Phoenix Stadium','Cleveland Browns Stadium',
                               'LP Field','Reliant Stadium','McAfee Coliseum','Monster Park','Texas Stadium',
                               'Invesco Field at Mile High','Qwest Field','Dolphin Stadium','Ralph Wilson Stadium',
                               'Louisiana Superdome','Jacksonville Municipal Stadium','Oakland-Alameda County Coliseum',
                               'Cowboys Stadium','New Meadowlands Stadium','EverBank Field','Mall of America Field',
                               'Sun Life Stadium','Sports Authority Field at Mile High','Mercedes-Benz Superdome',
                               'CenturyLink Field','O.co Coliseum','New Era Field','Bills Stadium',
                               'GEHA Field at Arrowhead Stadium'],
                              ['MetLife Stadium','TIAA Bank Stadium','State Farm Stadium','FirstEnergy Stadium',
                               'Nissan Stadium','NRG Stadium','Ring Central Coliseum','Candlestick Park','AT&T Stadium',
                               'Empower Field at Mile High','Lumen Field','Hard Rock Stadium','Highmark Stadium',
                               'Caesars Superdome','TIAA Bank Stadium','Ring Central Coliseum','AT&T Stadium',
                               'MetLife Stadium','TIAA Bank Stadium','Hubert H. Humphrey Metrodome',
                               'Hard Rock Stadium','Empower Field at Mile High','Caesars Superdome','Lumen Field',
                               'Ring Central Coliseum','Highmark Stadium','Highmark Stadium','Arrowhead Stadium'])

In [6]:
temps = []
humidities = []
winds = []

divisional = []
indoor = []

vegas_home_spreads = []
home_cov_spreads = []
o_u_results = []

matchup_home_cov_spreads = []
matchup_o_u_results = []


for i in range(len(game_data)):
    game_data.loc[i,'game_date'] = datetime.strptime(game_data['game_date'][i], '%m/%d/%Y')
    
    time = game_data.loc[i,'game_time'].split(':')
    game_data.loc[i,'game_time'] = int(time[0]) + (int(time[1][:2]) / 60) + 12*('pm' in game_data.loc[i,'game_time'])
    
    temps.append(int(game_data.loc[i,'weather'].split(' degrees')[0]))
    try:
        humidities.append(int(game_data.loc[i,'weather'].split('%')[0].split(' ')[-1]))
    except:
        humidities.append(60)
    if('no wind' in game_data.loc[i,'weather']):
        winds.append(0)
    else:
        winds.append(int(game_data.loc[i,'weather'].split(' mph')[0].split(' ')[-1]))
    
    
    if('Pick' in game_data.loc[i,'vegas_spread']):
        spread = 0.0
    else:
        spread = float(game_data.loc[i,'vegas_spread'].split('-')[-1])
        if(game_data.loc[i,'home_team'] in game_data.loc[i,'vegas_spread']):
            spread = -1*spread
    
    vegas_home_spreads.append(spread)
    
    
    if(game_data.loc[i, 'away_pts'] - game_data.loc[i, 'home_pts'] < spread):
        home_cov_spreads.append(1)
    elif(game_data.loc[i, 'away_pts'] - game_data.loc[i, 'home_pts'] > spread):
        home_cov_spreads.append(0)
    else:
        home_cov_spreads.append(randint(0,1))
        
    if(game_data.loc[i, 'away_pts'] + game_data.loc[i, 'home_pts'] > game_data.loc[i, 'vegas_o_u']):
        o_u_results.append(1)
    elif(game_data.loc[i, 'away_pts'] + game_data.loc[i, 'home_pts'] < game_data.loc[i, 'vegas_o_u']):
        o_u_results.append(0)
    else:
        o_u_results.append(randint(0,1))
    
    
    if(divisions[game_data.loc[i, 'home_team_code']] == divisions[game_data.loc[i, 'away_team_code']]):
        divisional.append(1)
    else:
        divisional.append(0)
        
        
    if(game_data.loc[i, 'stadium'] in indoor_stadiums):
        indoor.append(1)
    else:
        indoor.append(0)
        
    
    
for i in range(len(matchup_data)):
    matchup_data.loc[i,'game_date'] = datetime.strptime(matchup_data['game_date'][i], '%m/%d/%Y')
    
    if('Pick' in matchup_data.loc[i,'vegas_spread']):
        spread = 0.0
    else:
        spread = float(matchup_data.loc[i,'vegas_spread'].split('-')[-1])
        if(matchup_data.loc[i,'home_team'] in matchup_data.loc[i,'vegas_spread']):
            spread = -1*spread
            
    if(matchup_data.loc[i, 'away_pts'] - matchup_data.loc[i, 'home_pts'] < spread):
        matchup_home_cov_spreads.append(1)
    elif(matchup_data.loc[i, 'away_pts'] - matchup_data.loc[i, 'home_pts'] > spread):
        matchup_home_cov_spreads.append(0)
    else:
        matchup_home_cov_spreads.append(0.5)
        
    if(matchup_data.loc[i, 'away_pts'] + matchup_data.loc[i, 'home_pts'] > matchup_data.loc[i, 'vegas_o_u']):
        matchup_o_u_results.append(1)
    elif(matchup_data.loc[i, 'away_pts'] + matchup_data.loc[i, 'home_pts'] < matchup_data.loc[i, 'vegas_o_u']):
        matchup_o_u_results.append(0)
    else:
        matchup_o_u_results.append(0.5)
    

    
game_data['temp'] = temps
game_data['humidity'] = humidities
game_data['wind'] = winds
game_data = game_data.drop('weather', axis=1)

game_data['vegas_home_spread'] = vegas_home_spreads
game_data = game_data.drop('vegas_spread', axis=1)

game_data['home_covered_spread'] = home_cov_spreads
game_data['o_u_result'] = o_u_results

game_data['divisional_game'] = divisional
game_data['indoor_stadium'] = indoor


matchup_data['home_covered_spread'] = matchup_home_cov_spreads
matchup_data['o_u_result'] = matchup_o_u_results

In [7]:
# order dataframes by game_date (chronological)
game_data = game_data.sort_values('game_date', ascending=True).reset_index(drop=True)
matchup_data = matchup_data.sort_values('game_date', ascending=True).reset_index(drop=True)

# check for any missing values in data
print('Number of missing values in game_data: ' + str(sum(game_data.isnull().sum())))
print('Number of missing values in matchup_data: ' + str(sum(matchup_data.isnull().sum())))

Number of missing values in game_data: 0
Number of missing values in matchup_data: 0


In [8]:
# set coefficients (weights) of past games for calculating team stats
past_game_coef = [1/9, 1/9, 1/9, 1/9, 1/9, 1/9, 1/18, 1/18, 1/18, 1/18, 1/18, 1/18]
# additional weights of game stats for matching/unmatching home/away status to predction game 
match_visit_stat_coef = 0.6
non_match_visit_stat_coef = 0.4

In [9]:
# start_game_index: index row of game_data containing the first game with available team stats
start_game_index = len(past_game_coef) * 16

# num_total_games: number of games available with team stats
num_total_games = game_data.shape[0] - start_game_index

# avail_game_data: all games in game_data with team stats available
avail_game_data = game_data[start_game_index:].reset_index(drop=True)


print('Number of available games in data: ' + str(num_total_games))

Number of available games in data: 4078


In [10]:
# return a list of indexes corresponding to previous games in game_data that a team played before a given date
def get_previous_games_i(team, date):
    last_game = game_data.index[game_data['game_date']==date][0]
    previous_games = game_data[:last_game]
    previous_games_i = []
    for x in range(len(past_game_coef)):
        if(previous_games.index[previous_games['away_team_code']==team].tolist() == []):
            away_game = 0
        else:
            away_game = previous_games.index[previous_games['away_team_code']==team].tolist()[-1]
        
        if(previous_games.index[previous_games['home_team_code']==team].tolist() == []):
            home_game = 0
        else:
            home_game = previous_games.index[previous_games['home_team_code']==team].tolist()[-1]
        
        last_game = max(away_game, home_game)
        previous_games_i.append(last_game)
        previous_games = game_data[:last_game]
        
    return previous_games_i

In [11]:
# return a list of indexes corresponding to previous games in matchup_data between two given teams over the last 5 years
# (maximum of last 5 macthups returned)
def get_prev_matchups_i(team_1, team_2, date):
    last_game = matchup_data.index[matchup_data['game_date'] == date][0]
    furthest_game = last_game - 1335
    
    prev_games = matchup_data[furthest_game:last_game]
    prev_games_i = []
    
    count = 0
    searching = True
    
    while searching:
        if((prev_games.index[(prev_games['home_team_code']==team_1)&(prev_games['away_team_code']==team_2)].tolist() == []) 
           &(prev_games.index[(prev_games['home_team_code']==team_2)&(prev_games['away_team_code']==team_1)].tolist() == [])):
            searching = False
        else:
            if(prev_games.index[(prev_games['home_team_code']==team_1)&(prev_games['away_team_code']==team_2)].tolist() == []):
                game_1 = 0
            else:
                game_1 = \
                    prev_games.index[(prev_games['home_team_code']==team_1)&(prev_games['away_team_code']==team_2)].tolist()[-1]
        
            if(prev_games.index[(prev_games['home_team_code']==team_2)&(prev_games['away_team_code']==team_1)].tolist() == []):
                game_2 = 0
            else:
                game_2 = \
                    prev_games.index[(prev_games['home_team_code']==team_2)&(prev_games['away_team_code']==team_1)].tolist()[-1]
        
        if(searching):
            last_game = max(game_1, game_2)
            prev_games_i.append(last_game)
            prev_games = matchup_data[furthest_game:last_game]

            count += 1
            if (count >= 5):
                searching = False
        
    return prev_games_i

In [12]:
def fill_feature_1(feat1, feat2, feat3, feat4, stat_name_num, stat_name_den):
    home_off_divisor = 0
    home_def_divisor = 0
    away_off_divisor = 0
    away_def_divisor = 0
    
    home_off_past_game_coef = copy.copy(past_game_coef)
    home_def_past_game_coef = copy.copy(past_game_coef)
    away_off_past_game_coef = copy.copy(past_game_coef)
    away_def_past_game_coef = copy.copy(past_game_coef)
    
    home_off_stats = np.zeros((len(past_game_coef)))
    away_off_stats = np.zeros((len(past_game_coef)))
    home_def_stats = np.zeros((len(past_game_coef)))
    away_def_stats = np.zeros((len(past_game_coef)))
    
    for j in range(len(past_game_coef)):
        
        weight_sum = 0
        if(game_data['home_' + stat_name_den][h_games[j]] != 0):
            stat =  game_data['home_' + stat_name_num][h_games[j]] / game_data['home_' + stat_name_den][h_games[j]]
            if(game_data['home_team_code'][h_games[j]] == h_team):
                home_off_stats[j] = stat
            else:
                home_def_stats[j] = stat
        else:
            if(game_data['home_team_code'][h_games[j]] == h_team):
                home_off_stats[j] = 0
                home_off_past_game_coef[j] = 0
                for n in range(len(past_game_coef)):
                    weight_sum += home_off_past_game_coef[n]
                    
                if(weight_sum != 0):
                    for m in range(len(past_game_coef)):
                        home_off_past_game_coef[m] = home_off_past_game_coef[m] * (1 / weight_sum)
            else:
                home_def_stats[j] = 0
                home_def_past_game_coef[j] = 0
                for n in range(len(past_game_coef)):
                    weight_sum += home_def_past_game_coef[n]
            
                if(weight_sum != 0):
                    for m in range(len(past_game_coef)):
                        home_def_past_game_coef[m] = home_def_past_game_coef[m] * (1 / weight_sum)
                        
        weight_sum = 0
        if(game_data['away_' + stat_name_den][h_games[j]] != 0):
            stat =  game_data['away_' + stat_name_num][h_games[j]] / game_data['away_' + stat_name_den][h_games[j]]
            if(game_data['away_team_code'][h_games[j]] == h_team):
                home_off_stats[j] = stat
            else:
                home_def_stats[j] = stat
        else:
            if(game_data['away_team_code'][h_games[j]] == h_team):
                home_off_stats[j] = 0
                home_off_past_game_coef[j] = 0
                for n in range(len(past_game_coef)):
                    weight_sum += home_off_past_game_coef[n]
                    
                if(weight_sum != 0):
                    for m in range(len(past_game_coef)):
                        home_off_past_game_coef[m] = home_off_past_game_coef[m] * (1 / weight_sum)
            else:
                home_def_stats[j] = 0
                home_def_past_game_coef[j] = 0
                for n in range(len(past_game_coef)):
                    weight_sum += home_def_past_game_coef[n]
            
                if(weight_sum != 0):
                    for m in range(len(past_game_coef)):
                        home_def_past_game_coef[m] = home_def_past_game_coef[m] * (1 / weight_sum)
                
        weight_sum = 0
        if(game_data['home_' + stat_name_den][a_games[j]] != 0):
            stat =  game_data['home_' + stat_name_num][a_games[j]] / game_data['home_' + stat_name_den][a_games[j]]
            if(game_data['home_team_code'][a_games[j]] == a_team):
                away_off_stats[j] = stat
            else:
                away_def_stats[j] = stat
        else:
            if(game_data['home_team_code'][a_games[j]] == a_team):
                away_off_stats[j] = 0
                away_off_past_game_coef[j] = 0
                for n in range(len(past_game_coef)):
                    weight_sum += away_off_past_game_coef[n]
                    
                if(weight_sum != 0):
                    for m in range(len(past_game_coef)):
                        away_off_past_game_coef[m] = away_off_past_game_coef[m] * (1 / weight_sum)
            else:
                away_def_stats[j] = 0
                away_def_past_game_coef[j] = 0
                for n in range(len(past_game_coef)):
                    weight_sum += away_def_past_game_coef[n]
            
                if(weight_sum != 0):
                    for m in range(len(past_game_coef)):
                        away_def_past_game_coef[m] = away_def_past_game_coef[m] * (1 / weight_sum)
            
        weight_sum = 0
        if(game_data['away_' + stat_name_den][a_games[j]] != 0):
            stat =  game_data['away_' + stat_name_num][a_games[j]] / game_data['away_' + stat_name_den][a_games[j]]
            if(game_data['away_team_code'][a_games[j]] == a_team):
                away_off_stats[j] = stat
            else:
                away_def_stats[j] = stat
        else:
            if(game_data['away_team_code'][a_games[j]] == a_team):
                away_off_stats[j] = 0
                away_off_past_game_coef[j] = 0
                for n in range(len(past_game_coef)):
                    weight_sum += away_off_past_game_coef[n]
                    
                if(weight_sum != 0):
                    for m in range(len(past_game_coef)):
                        away_off_past_game_coef[m] = away_off_past_game_coef[m] * (1 / weight_sum)
            else:
                away_def_stats[j] = 0
                away_def_past_game_coef[j] = 0
                for n in range(len(past_game_coef)):
                    weight_sum += away_def_past_game_coef[n]
            
                if(weight_sum != 0):
                    for m in range(len(past_game_coef)):
                        away_def_past_game_coef[m] = away_def_past_game_coef[m] * (1 / weight_sum)
                        
    for j in range(len(past_game_coef)):
        if(game_data['home_team_code'][h_games[j]] == h_team):
            feat1[i] += home_off_past_game_coef[j]*match_visit_stat_coef*home_off_stats[j]
            feat3[i] += home_def_past_game_coef[j]*match_visit_stat_coef*home_def_stats[j]
            home_off_divisor += home_off_past_game_coef[j]*match_visit_stat_coef
            home_def_divisor += home_def_past_game_coef[j]*match_visit_stat_coef
        else:
            feat1[i] += home_off_past_game_coef[j]*non_match_visit_stat_coef*home_off_stats[j]
            feat3[i] += home_def_past_game_coef[j]*non_match_visit_stat_coef*home_def_stats[j]
            home_off_divisor += home_off_past_game_coef[j]*non_match_visit_stat_coef
            home_def_divisor += home_def_past_game_coef[j]*non_match_visit_stat_coef
            
        if(game_data['home_team_code'][a_games[j]] == a_team):
            feat2[i] += away_off_past_game_coef[j]*non_match_visit_stat_coef*away_off_stats[j]
            feat4[i] += away_def_past_game_coef[j]*non_match_visit_stat_coef*away_def_stats[j]
            away_off_divisor += away_off_past_game_coef[j]*non_match_visit_stat_coef
            away_def_divisor += away_def_past_game_coef[j]*non_match_visit_stat_coef
        else:
            feat2[i] += away_off_past_game_coef[j]*match_visit_stat_coef*away_off_stats[j]
            feat4[i] += away_def_past_game_coef[j]*match_visit_stat_coef*away_def_stats[j]
            away_off_divisor += away_off_past_game_coef[j]*match_visit_stat_coef
            away_def_divisor += away_def_past_game_coef[j]*match_visit_stat_coef
    
    if(home_off_divisor == 0):
        feat1[i] = 0
    else:
        feat1[i] = feat1[i] / home_off_divisor
    if(away_off_divisor == 0):
        feat2[i] = 0
    else:
        feat2[i] = feat2[i] / away_off_divisor
    if(home_def_divisor == 0):
        feat3[i] = 0
    else:
        feat3[i] = feat3[i] / home_def_divisor
    if(away_def_divisor == 0):
        feat4[i] = 0
    else:
        feat4[i] = feat4[i] / away_def_divisor
    

In [13]:
def fill_feature_2(feat1, feat2, stat_name_num, stat_name_den):
    h_divisor = 0
    a_divisor = 0
    
    h_past_game_coef = copy.copy(past_game_coef)
    a_past_game_coef = copy.copy(past_game_coef)
    
    h_home_stats = np.zeros((len(past_game_coef)))
    h_away_stats = np.zeros((len(past_game_coef)))
    a_home_stats = np.zeros((len(past_game_coef)))
    a_away_stats = np.zeros((len(past_game_coef)))
    
    for j in range(len(past_game_coef)):
        
        weight_sum = 0
        if((game_data['home_team_code'][h_games[j]] == h_team) and (game_data['home_' + stat_name_den][h_games[j]] != 0)):
            h_home_stats[j] = game_data['home_' + stat_name_num][h_games[j]] / game_data['home_' + stat_name_den][h_games[j]]
        elif(game_data['home_team_code'][h_games[j]] == h_team):
            h_home_stats[j] = 0
            h_past_game_coef[j] = 0
            for n in range(len(past_game_coef)):
                weight_sum += h_past_game_coef[n]
            
            if(weight_sum != 0):
                for m in range(len(past_game_coef)):
                    h_past_game_coef[m] = h_past_game_coef[m] * (1 / weight_sum)
        
        weight_sum = 0
        if((game_data['away_team_code'][h_games[j]] == h_team) and (game_data['away_' + stat_name_den][h_games[j]] != 0)):
            h_away_stats[j] = game_data['away_' + stat_name_num][h_games[j]] / game_data['away_' + stat_name_den][h_games[j]]
        elif(game_data['away_team_code'][h_games[j]] == h_team):
            h_away_stats[j] = 0
            h_past_game_coef[j] = 0
            for n in range(len(past_game_coef)):
                weight_sum += h_past_game_coef[n]
                
            if(weight_sum != 0):
                for m in range(len(past_game_coef)):
                    h_past_game_coef[m] = h_past_game_coef[m] * (1 / weight_sum)
                
        weight_sum = 0
        if((game_data['home_team_code'][a_games[j]] == a_team) and (game_data['home_' + stat_name_den][a_games[j]] != 0)):
            a_home_stats[j] = game_data['home_' + stat_name_num][a_games[j]] / game_data['home_' + stat_name_den][a_games[j]]
        elif(game_data['home_team_code'][a_games[j]] == a_team):
            a_home_stats[j] = 0
            a_past_game_coef[j] = 0
            for n in range(len(past_game_coef)):
                weight_sum += a_past_game_coef[n]
                
            if(weight_sum != 0):
                for m in range(len(past_game_coef)):
                    a_past_game_coef[m] = a_past_game_coef[m] * (1 / weight_sum)
                
        weight_sum = 0
        if((game_data['away_team_code'][a_games[j]] == a_team) and (game_data['away_' + stat_name_den][a_games[j]] != 0)):
            a_away_stats[j] = game_data['away_' + stat_name_num][a_games[j]] / game_data['away_' + stat_name_den][a_games[j]]
        elif(game_data['away_team_code'][a_games[j]] == a_team):
            a_away_stats[j] = 0
            a_past_game_coef[j] = 0
            for n in range(len(past_game_coef)):
                weight_sum += a_past_game_coef[n]
            
            if(weight_sum != 0):
                for m in range(len(past_game_coef)):
                    a_past_game_coef[m] = a_past_game_coef[m] * (1 / weight_sum)
                
    for j in range(len(past_game_coef)):
        if(game_data['home_team_code'][h_games[j]] == h_team):
            feat1[i] += h_past_game_coef[j]*match_visit_stat_coef*h_home_stats[j]
            h_divisor += h_past_game_coef[j]*match_visit_stat_coef
        else:
            feat1[i] += h_past_game_coef[j]*non_match_visit_stat_coef*h_away_stats[j]
            h_divisor += h_past_game_coef[j]*non_match_visit_stat_coef
            
        if(game_data['home_team_code'][a_games[j]] == a_team):
            feat2[i] += a_past_game_coef[j]*non_match_visit_stat_coef*a_home_stats[j]
            a_divisor += a_past_game_coef[j]*non_match_visit_stat_coef
        else:
            feat2[i] += a_past_game_coef[j]*match_visit_stat_coef*a_away_stats[j]
            a_divisor += a_past_game_coef[j]*match_visit_stat_coef
    
    if(h_divisor == 0):
        feat1[i] = 0
    else:
        feat1[i] = feat1[i] / h_divisor
        
    if(a_divisor == 0):
        feat2[i] = 0
    else:
        feat2[i] = feat2[i] / a_divisor

In [14]:
# create general game features

# binary: 1 if game played in indoor stadium, 0 otherwise
indoor_stadium = avail_game_data[['indoor_stadium']].to_numpy()

# binary: 1 if game played between teams in the same division, 0 otherwise
divisional = avail_game_data[['divisional_game']].to_numpy()

# one-hot encoded: for each feature, 1 if game played in a specific stadium, 0 otherwise
stadium = avail_game_data[['stadium']].to_numpy()

# game start time
game_time = avail_game_data[['game_time']].to_numpy()

# one-hot encoded: referee
referee = avail_game_data[['referee']].to_numpy()

# game temperature (70 if indoor)
temp = avail_game_data[['temp']].to_numpy()

# game humidity (0.45 if indoor)
humidity = avail_game_data[['humidity']].to_numpy()

# game wind speed (0 if indoor)
wind_speed = avail_game_data[['wind']].to_numpy()

# create point total column: home pts + away pts
point_total = avail_game_data[['home_pts']].to_numpy() + avail_game_data[['away_pts']].to_numpy()

# vegas over-under line
vegas_o_u = avail_game_data[['vegas_o_u']].to_numpy()

# binary: 1 if point total hit the vegas over, 0 if total hit under, and randomly 0 or 1 if the total tied vegas line
o_u_result = avail_game_data[['o_u_result']].to_numpy()

# vegas spread (in terms of home team)
vegas_home_spread = avail_game_data[['vegas_home_spread']].to_numpy()

# binary: 1 if home team covered the vegas spread, 0 if away team covered vegas spread, and randomly 0 or 1 if neither occurred
home_covered_spread = avail_game_data[['home_covered_spread']].to_numpy()

# number of times a game between these two teams hit the vegas over during the past 5 years (max of last 5 games)
team_matchups_over = np.zeros((len(avail_game_data), 1))
# number of times a game between these two teams hit the vegas under during the past 5 years (max of last 5 games)
team_matchups_under = np.zeros((len(avail_game_data), 1))

# num times the current home team covered vegas spread in games b/w these 2 teams during the past 5 years (max of last 5 games)
team_matchups_h_covered = np.zeros((len(avail_game_data), 1))
# num times the current away team covered vegas spread in games b/w these 2 teams during the past 5 years (max of last 5 games)
team_matchups_a_covered = np.zeros((len(avail_game_data), 1))

for i in range(len(avail_game_data)):
    inds = get_prev_matchups_i(avail_game_data.loc[i,'home_team_code'], avail_game_data.loc[i,'away_team_code'], 
                               avail_game_data.loc[i,'game_date'])
    if(inds != []):
        for j in inds:
            if(matchup_data.loc[j,'o_u_result'] == 1):
                team_matchups_over[i] += 1
            elif(matchup_data.loc[j,'o_u_result'] == 0):
                team_matchups_under[i] += 1
            
            if(avail_game_data.loc[i,'home_team_code'] == matchup_data.loc[j,'home_team_code']):
                if(matchup_data.loc[j,'home_covered_spread'] == 1):
                    team_matchups_h_covered[i] += 1
                elif(matchup_data.loc[j,'home_covered_spread'] == 0):
                    team_matchups_a_covered[i] += 1
            else:
                if(matchup_data.loc[j,'home_covered_spread'] == 1):
                    team_matchups_a_covered[i] += 1
                elif(matchup_data.loc[j,'home_covered_spread'] == 0):
                    team_matchups_h_covered[i] += 1
        

In [15]:
# team features

# name of team head coach
home_coach = avail_game_data[['home_coach']].to_numpy()
away_coach = avail_game_data[['away_coach']].to_numpy()

# number of days since team's last game
home_rest_days = np.zeros((len(avail_game_data),1))
away_rest_days = np.zeros((len(avail_game_data),1))

# team average time of possession (not including overtime)
home_pos_time = np.zeros((len(avail_game_data),1))
away_pos_time = np.zeros((len(avail_game_data),1))

# team points per game
home_off_pts_per_game = np.zeros((len(avail_game_data),1))
away_off_pts_per_game = np.zeros((len(avail_game_data),1))

# team points allowed per game
home_def_pts_per_game = np.zeros((len(avail_game_data),1))
away_def_pts_per_game = np.zeros((len(avail_game_data),1))

# team points per second half of game
home_off_pts_per_sec_half = np.zeros((len(avail_game_data),1))
away_off_pts_per_sec_half = np.zeros((len(avail_game_data),1))

# team points allowed per second half of game
home_def_pts_per_sec_half = np.zeros((len(avail_game_data),1))
away_def_pts_per_sec_half = np.zeros((len(avail_game_data),1))

# team points per play
home_off_pts_per_play = np.zeros((len(avail_game_data),1))
away_off_pts_per_play = np.zeros((len(avail_game_data),1))

# team points allowed per play
home_def_pts_per_play = np.zeros((len(avail_game_data),1))
away_def_pts_per_play = np.zeros((len(avail_game_data),1))

# team yards per play
home_off_yds_per_play = np.zeros((len(avail_game_data),1))
away_off_yds_per_play = np.zeros((len(avail_game_data),1))

# team yards allowed per play
home_def_yds_per_play = np.zeros((len(avail_game_data),1))
away_def_yds_per_play = np.zeros((len(avail_game_data),1))

# team time per play
home_off_time_per_play = np.zeros((len(avail_game_data),1))
away_off_time_per_play = np.zeros((len(avail_game_data),1))

# team time allowed per play
home_def_time_per_play = np.zeros((len(avail_game_data),1))
away_def_time_per_play = np.zeros((len(avail_game_data),1))

# team % of plays rushing
home_off_perc_rush = np.zeros((len(avail_game_data),1))
away_off_perc_rush = np.zeros((len(avail_game_data),1))

# team % of plays defending rush
home_def_perc_rush = np.zeros((len(avail_game_data),1))
away_def_perc_rush = np.zeros((len(avail_game_data),1))

# team % of first downs rushing
home_off_perc_fstdown_rush = np.zeros((len(avail_game_data),1))
away_off_perc_fstdown_rush = np.zeros((len(avail_game_data),1))

# team % of first downs defended rushing
home_def_perc_fstdown_rush = np.zeros((len(avail_game_data),1))
away_def_perc_fstdown_rush = np.zeros((len(avail_game_data),1))

# team explosive play rate: (16+ yard pass plays + 12+ yard rush plays) / (passing attempts + sacks taken + rush plays)
# Note: not an explosive play if does not convert on a third or fourth down
home_off_epr = np.zeros((len(avail_game_data),1))
away_off_epr = np.zeros((len(avail_game_data),1))

# team explosive play rate allowed: (16+ yard pass plays + 12+ yard rush plays) / (passing attempts + sacks taken + rush plays)
# Note: not an explosive play if does not convert on a third or fourth down
home_def_epr = np.zeros((len(avail_game_data),1))
away_def_epr = np.zeros((len(avail_game_data),1))

# team fumbles lost rate
home_off_fumble_rate = np.zeros((len(avail_game_data),1))
away_off_fumble_rate = np.zeros((len(avail_game_data),1))

# team fumbles taken rate
home_def_fumble_rate = np.zeros((len(avail_game_data),1))
away_def_fumble_rate = np.zeros((len(avail_game_data),1))

# team interceptions thrown rate
home_off_int_rate = np.zeros((len(avail_game_data),1))
away_off_int_rate = np.zeros((len(avail_game_data),1))

# team interceptions caught rate
home_def_int_rate = np.zeros((len(avail_game_data),1))
away_def_int_rate = np.zeros((len(avail_game_data),1))

# team sacks taken rate
home_off_sack_rate = np.zeros((len(avail_game_data),1))
away_off_sack_rate = np.zeros((len(avail_game_data),1))

# team sacks inflicted rate
home_def_sack_rate = np.zeros((len(avail_game_data),1))
away_def_sack_rate = np.zeros((len(avail_game_data),1))

# team turnover margin (takeaways - turnovers)
home_turnover_margin = np.zeros((len(avail_game_data),1))
away_turnover_margin = np.zeros((len(avail_game_data),1))

# team turnover margin (takeaways - turnovers) in their last game
home_lw_turnover_margin = np.zeros((len(avail_game_data),1))
away_lw_turnover_margin = np.zeros((len(avail_game_data),1))

# team third down conversion rate
home_off_thrdown_conv = np.zeros((len(avail_game_data),1))
away_off_thrdown_conv = np.zeros((len(avail_game_data),1))

# team third down conversion rate allowed
home_def_thrdown_conv = np.zeros((len(avail_game_data),1))
away_def_thrdown_conv = np.zeros((len(avail_game_data),1))

# team % of fourth downs attempted
home_off_perc_frthdown_att = np.zeros((len(avail_game_data),1))
away_off_perc_frthdown_att = np.zeros((len(avail_game_data),1))

# team % of opponent fourth downs attempted
home_def_perc_frthdown_att = np.zeros((len(avail_game_data),1))
away_def_perc_frthdown_att = np.zeros((len(avail_game_data),1))

# team fourth down conversion rate
home_off_frthdown_conv = np.zeros((len(avail_game_data),1))
away_off_frthdown_conv = np.zeros((len(avail_game_data),1))

# team fourth down conversion rate allowed
home_def_frthdown_conv = np.zeros((len(avail_game_data),1))
away_def_frthdown_conv = np.zeros((len(avail_game_data),1))

# team red zone efficiency
home_off_rze = np.zeros((len(avail_game_data),1))
away_off_rze = np.zeros((len(avail_game_data),1))

# team red zone efficiency allowed
home_def_rze = np.zeros((len(avail_game_data),1))
away_def_rze = np.zeros((len(avail_game_data),1))

# team PAT success %
home_pat = np.zeros((len(avail_game_data),1))
away_pat = np.zeros((len(avail_game_data),1))

# team field goal success % from 0-39 yards
home_FG_39 = np.zeros((len(avail_game_data),1))
away_FG_39 = np.zeros((len(avail_game_data),1))

# team field goal success % from 40-49 yards
home_FG_49 = np.zeros((len(avail_game_data),1))
away_FG_49 = np.zeros((len(avail_game_data),1))

# team field goal success % from 50+ yards
home_FG_50 = np.zeros((len(avail_game_data),1))
away_FG_50 = np.zeros((len(avail_game_data),1))

# team yards per punt
home_yds_per_punt = np.zeros((len(avail_game_data),1))
away_yds_per_punt = np.zeros((len(avail_game_data),1))

# team % punts inside opponent 20-yard line
home_punts_in_20 = np.zeros((len(avail_game_data),1))
away_punts_in_20 = np.zeros((len(avail_game_data),1))

# team points allowed per 100 yards
home_def_pts_100 = np.zeros((len(avail_game_data),1))
away_def_pts_100 = np.zeros((len(avail_game_data),1))

# team fourth down conversion rate
home_off_2pt_conv = np.zeros((len(avail_game_data),1))
away_off_2pt_conv = np.zeros((len(avail_game_data),1))
    
# team fourth down conversion rate allowed
home_def_2pt_conv = np.zeros((len(avail_game_data),1))
away_def_2pt_conv = np.zeros((len(avail_game_data),1))

# team % of two-pt conversions attempted
home_off_perc_2pt_att = np.zeros((len(avail_game_data),1))
away_off_perc_2pt_att = np.zeros((len(avail_game_data),1))

# team % of opponent two-pt conversions attempted
home_def_perc_2pt_att = np.zeros((len(avail_game_data),1))
away_def_perc_2pt_att = np.zeros((len(avail_game_data),1))

# team yards per punt return
home_off_yds_per_punt_return = np.zeros((len(avail_game_data),1))
away_off_yds_per_punt_return = np.zeros((len(avail_game_data),1))

# team yards per punt return allowed
home_def_yds_per_punt_return = np.zeros((len(avail_game_data),1))
away_def_yds_per_punt_return = np.zeros((len(avail_game_data),1))

# team % kickoffs received that are returned
home_off_perc_kickoffs_returned = np.zeros((len(avail_game_data),1))
away_off_perc_kickoffs_returned = np.zeros((len(avail_game_data),1))

# team % kickoffs kicked that are returned
home_def_perc_kickoffs_returned = np.zeros((len(avail_game_data),1))
away_def_perc_kickoffs_returned = np.zeros((len(avail_game_data),1))

# team yards per kickoff return
home_off_yds_per_kickoff_return = np.zeros((len(avail_game_data),1))
away_off_yds_per_kickoff_return = np.zeros((len(avail_game_data),1))

# team yards per kickoff return allowed
home_def_yds_per_kickoff_return = np.zeros((len(avail_game_data),1))
away_def_yds_per_kickoff_return = np.zeros((len(avail_game_data),1))

# team short pass completion rate
home_off_short_pass_compl_rate = np.zeros((len(avail_game_data),1))
away_off_short_pass_compl_rate = np.zeros((len(avail_game_data),1))

# team short pass completion rate allowed
home_def_short_pass_compl_rate = np.zeros((len(avail_game_data),1))
away_def_short_pass_compl_rate = np.zeros((len(avail_game_data),1))

# team deep pass completion rate
home_off_deep_pass_compl_rate = np.zeros((len(avail_game_data),1))
away_off_deep_pass_compl_rate = np.zeros((len(avail_game_data),1))

# team deep pass completion rate allowed
home_def_deep_pass_compl_rate = np.zeros((len(avail_game_data),1))
away_def_deep_pass_compl_rate = np.zeros((len(avail_game_data),1))

# team passes to middle completion rate
home_off_pass_compl_rate_middle = np.zeros((len(avail_game_data),1))
away_off_pass_compl_rate_middle = np.zeros((len(avail_game_data),1))

# team passes to middle completion rate allowed
home_def_pass_compl_rate_middle = np.zeros((len(avail_game_data),1))
away_def_pass_compl_rate_middle = np.zeros((len(avail_game_data),1))

# team offensive average starting field position
home_off_avg_starting_field_pos = np.zeros((len(avail_game_data),1))
away_off_avg_starting_field_pos = np.zeros((len(avail_game_data),1))

# team defensive average starting field position
home_def_avg_starting_field_pos = np.zeros((len(avail_game_data),1))
away_def_avg_starting_field_pos = np.zeros((len(avail_game_data),1))

# team offensive penalty yards per game
home_off_pen_yds = np.zeros((len(avail_game_data),1))
away_off_pen_yds = np.zeros((len(avail_game_data),1))

# team defensive penalty yards per game
home_def_pen_yds = np.zeros((len(avail_game_data),1))
away_def_pen_yds = np.zeros((len(avail_game_data),1))

# team opponent offensive penalty yards per game
home_opp_off_pen_yds = np.zeros((len(avail_game_data),1))
away_opp_off_pen_yds = np.zeros((len(avail_game_data),1))

# team opponent defensive penalty yards per game
home_opp_def_pen_yds = np.zeros((len(avail_game_data),1))
away_opp_def_pen_yds = np.zeros((len(avail_game_data),1))

# team rush yards per carry
home_off_rush_ypc = np.zeros((len(avail_game_data),1))
away_off_rush_ypc = np.zeros((len(avail_game_data),1))

# team rush yards per carry allowed
home_def_rush_ypc = np.zeros((len(avail_game_data),1))
away_def_rush_ypc = np.zeros((len(avail_game_data),1))

# team % rush plays to end
home_off_perc_rush_ends = np.zeros((len(avail_game_data),1))
away_off_perc_rush_ends = np.zeros((len(avail_game_data),1))

# team opponent % rush plays to end
home_def_perc_rush_ends = np.zeros((len(avail_game_data),1))
away_def_perc_rush_ends = np.zeros((len(avail_game_data),1))

# team early-down rush success rate
home_off_early_down_rush_suc = np.zeros((len(avail_game_data),1))
away_off_early_down_rush_suc = np.zeros((len(avail_game_data),1))

# team early-down rush success rate allowed
home_def_early_down_rush_suc = np.zeros((len(avail_game_data),1))
away_def_early_down_rush_suc = np.zeros((len(avail_game_data),1))

# team early-down pass success rate
home_off_early_down_pass_suc = np.zeros((len(avail_game_data),1))
away_off_early_down_pass_suc = np.zeros((len(avail_game_data),1))

# team early-down pass success rate allowed
home_def_early_down_pass_suc = np.zeros((len(avail_game_data),1))
away_def_early_down_pass_suc = np.zeros((len(avail_game_data),1))

# team passes to middle completion rate
home_off_perc_pass_middle = np.zeros((len(avail_game_data),1))
away_off_perc_pass_middle = np.zeros((len(avail_game_data),1))

# team passes to middle completion rate allowed
home_def_perc_pass_middle = np.zeros((len(avail_game_data),1))
away_def_perc_pass_middle = np.zeros((len(avail_game_data),1))

# team pass completion rate
home_off_pass_compl_rate = np.zeros((len(avail_game_data),1))
away_off_pass_compl_rate = np.zeros((len(avail_game_data),1))

# team pass completion rate allowed
home_def_pass_compl_rate = np.zeros((len(avail_game_data),1))
away_def_pass_compl_rate = np.zeros((len(avail_game_data),1))

# team adjusted net yards per passing attempt: 
# (gross pass yds + 20*(pass TDs) - 45*(ints thrown) - sack yds taken) / (passing attempts + sacks taken)
home_off_adj_net_yds_per_att = np.zeros((len(avail_game_data),1))
away_off_adj_net_yds_per_att = np.zeros((len(avail_game_data),1))

# team adjusted net yards per passing attempt allowed: 
# (gross pass yds + 20*(pass TDs) - 45*(ints thrown) - sack yds taken) / (passing attempts + sacks taken)
home_def_adj_net_yds_per_att = np.zeros((len(avail_game_data),1))
away_def_adj_net_yds_per_att = np.zeros((len(avail_game_data),1))

# team season information
home_margin_o_u = np.zeros((len(avail_game_data),1))
away_margin_o_u = np.zeros((len(avail_game_data),1))
home_margin_ats = np.zeros((len(avail_game_data),1))
away_margin_ats = np.zeros((len(avail_game_data),1))
home_record_margin = np.zeros((len(avail_game_data),1))
away_record_margin = np.zeros((len(avail_game_data),1))
home_pt_margin_o_u = np.zeros((len(avail_game_data),1))
away_pt_margin_o_u = np.zeros((len(avail_game_data),1))
home_pt_margin_ats = np.zeros((len(avail_game_data),1))
away_pt_margin_ats = np.zeros((len(avail_game_data),1))
home_pt_margin = np.zeros((len(avail_game_data),1))
away_pt_margin = np.zeros((len(avail_game_data),1))


for i in range(len(avail_game_data)):
    h_team = avail_game_data['home_team_code'][i]
    a_team = avail_game_data['away_team_code'][i]
    game_date = avail_game_data['game_date'][i]
    
    h_games = get_previous_games_i(h_team, game_date)
    a_games = get_previous_games_i(a_team, game_date)
    
    previous_games = game_data[:start_game_index + i]
    h_last_away_game_i = previous_games.index[previous_games['away_team_code']==h_team].tolist()[-1]
    h_last_home_game_i = previous_games.index[previous_games['home_team_code']==h_team].tolist()[-1]
    h_last_game_i = max(h_last_away_game_i, h_last_home_game_i)
    a_last_away_game_i = previous_games.index[previous_games['away_team_code']==a_team].tolist()[-1]
    a_last_home_game_i = previous_games.index[previous_games['home_team_code']==a_team].tolist()[-1]
    a_last_game_i = max(a_last_away_game_i, a_last_home_game_i)
    
    h_last_game_date = game_data['game_date'][h_last_game_i]
    home_rest_days[i] = min(21, (game_date - h_last_game_date).days)
    a_last_game_date = game_data['game_date'][a_last_game_i]
    away_rest_days[i] = min(21, (game_date - a_last_game_date).days)
    
    h_divisor = 0
    a_divisor = 0
    for j in range(len(past_game_coef)):
        if(game_data['home_team_code'][h_games[j]] == h_team):
            home_pos_time[i] += past_game_coef[j]*match_visit_stat_coef*game_data['home_pos_time'][h_games[j]]
            h_divisor += past_game_coef[j]*match_visit_stat_coef
        else:
            home_pos_time[i] += past_game_coef[j]*non_match_visit_stat_coef*game_data['away_pos_time'][h_games[j]]
            h_divisor += past_game_coef[j]*non_match_visit_stat_coef
            
        if(game_data['home_team_code'][a_games[j]] == a_team):
            away_pos_time[i] += past_game_coef[j]*non_match_visit_stat_coef*game_data['home_pos_time'][a_games[j]]
            a_divisor += past_game_coef[j]*non_match_visit_stat_coef
        else:
            away_pos_time[i] += past_game_coef[j]*match_visit_stat_coef*game_data['away_pos_time'][a_games[j]]
            a_divisor += past_game_coef[j]*match_visit_stat_coef
    
    home_pos_time[i] = home_pos_time[i] / h_divisor
    away_pos_time[i] = away_pos_time[i] / a_divisor
    
    
    h_divisor = 0
    a_divisor = 0
    for j in range(len(past_game_coef)):
        if(game_data['home_team_code'][h_games[j]] == h_team):
            home_off_pts_per_game[i] += past_game_coef[j]*match_visit_stat_coef*game_data['home_pts'][h_games[j]]
            home_def_pts_per_game[i] += past_game_coef[j]*match_visit_stat_coef*game_data['away_pts'][h_games[j]]
            h_divisor += past_game_coef[j]*match_visit_stat_coef
        else:
            home_off_pts_per_game[i] += past_game_coef[j]*non_match_visit_stat_coef*game_data['away_pts'][h_games[j]]
            home_def_pts_per_game[i] += past_game_coef[j]*non_match_visit_stat_coef*game_data['home_pts'][h_games[j]]
            h_divisor += past_game_coef[j]*non_match_visit_stat_coef
            
        if(game_data['home_team_code'][a_games[j]] == a_team):
            away_off_pts_per_game[i] += past_game_coef[j]*non_match_visit_stat_coef*game_data['home_pts'][a_games[j]]
            away_def_pts_per_game[i] += past_game_coef[j]*non_match_visit_stat_coef*game_data['away_pts'][a_games[j]]
            a_divisor += past_game_coef[j]*non_match_visit_stat_coef
        else:
            away_off_pts_per_game[i] += past_game_coef[j]*match_visit_stat_coef*game_data['away_pts'][a_games[j]]
            away_def_pts_per_game[i] += past_game_coef[j]*match_visit_stat_coef*game_data['home_pts'][a_games[j]]
            a_divisor += past_game_coef[j]*match_visit_stat_coef
    
    home_off_pts_per_game[i] = home_off_pts_per_game[i] / h_divisor
    away_off_pts_per_game[i] = away_off_pts_per_game[i] / a_divisor
    home_def_pts_per_game[i] = home_def_pts_per_game[i] / h_divisor
    away_def_pts_per_game[i] = away_def_pts_per_game[i] / a_divisor
    
        
    h_divisor = 0
    a_divisor = 0
    for j in range(len(past_game_coef)):
        if(game_data['home_team_code'][h_games[j]] == h_team):
            home_off_pts_per_sec_half[i]+=past_game_coef[j]*match_visit_stat_coef*(game_data['home_q3_pts'][h_games[j]]
                                                                                + game_data['home_q4_pts'][h_games[j]])
            home_def_pts_per_sec_half[i]+=past_game_coef[j]*match_visit_stat_coef*(game_data['away_q3_pts'][h_games[j]]
                                                                                   + game_data['away_q4_pts'][h_games[j]])
            h_divisor += past_game_coef[j]*match_visit_stat_coef
        else:
            home_off_pts_per_sec_half[i]+=past_game_coef[j]*non_match_visit_stat_coef*(game_data['away_q3_pts'][h_games[j]]
                                                                                    + game_data['away_q4_pts'][h_games[j]])
            home_def_pts_per_sec_half[i]+=past_game_coef[j]*non_match_visit_stat_coef*(game_data['home_q3_pts'][h_games[j]]
                                                                                    + game_data['home_q4_pts'][h_games[j]])
            h_divisor += past_game_coef[j]*non_match_visit_stat_coef
            
        if(game_data['home_team_code'][a_games[j]] == a_team):
            away_off_pts_per_sec_half[i]+=past_game_coef[j]*non_match_visit_stat_coef*(game_data['home_q3_pts'][a_games[j]]
                                                                                 + game_data['home_q4_pts'][a_games[j]])
            away_def_pts_per_sec_half[i]+=past_game_coef[j]*non_match_visit_stat_coef*(game_data['away_q3_pts'][a_games[j]]
                                                                                    + game_data['away_q4_pts'][a_games[j]])
            a_divisor += past_game_coef[j]*non_match_visit_stat_coef
        else:
            away_off_pts_per_sec_half[i]+=past_game_coef[j]*match_visit_stat_coef*(game_data['away_q3_pts'][a_games[j]]
                                                                                + game_data['away_q4_pts'][a_games[j]])
            away_def_pts_per_sec_half[i]+=past_game_coef[j]*match_visit_stat_coef*(game_data['home_q3_pts'][a_games[j]]
                                                                                + game_data['home_q4_pts'][a_games[j]])
            a_divisor += past_game_coef[j]*match_visit_stat_coef
    
    home_off_pts_per_sec_half[i] = home_off_pts_per_sec_half[i] / h_divisor
    away_off_pts_per_sec_half[i] = away_off_pts_per_sec_half[i] / a_divisor
    home_def_pts_per_sec_half[i] = home_def_pts_per_sec_half[i] / h_divisor
    away_def_pts_per_sec_half[i] = away_def_pts_per_sec_half[i] / a_divisor
    
    
    h_divisor = 0
    a_divisor = 0
    for j in range(len(past_game_coef)):
        h_home_pts_j = (6*game_data['home_rush_tds'][h_games[j]] + 6*game_data['home_pass_tds'][h_games[j]] 
                        + game_data['home_pat_m'][h_games[j]] + 2*game_data['home_2pt_conv_suc'][h_games[j]] 
                        + 3*game_data['home_fgm_39'][h_games[j]] + 3*game_data['home_fgm_40_49'][h_games[j]] 
                        + 3*game_data['home_fgm_50'][h_games[j]]) / (game_data['home_rush_plays'][h_games[j]] 
                                                                     + game_data['home_pass_att'][h_games[j]] 
                                                                     + game_data['home_sacks_taken'][h_games[j]])
        
        h_away_pts_j = (6*game_data['away_rush_tds'][h_games[j]] + 6*game_data['away_pass_tds'][h_games[j]] 
                        + game_data['away_pat_m'][h_games[j]] + 2*game_data['away_2pt_conv_suc'][h_games[j]] 
                        + 3*game_data['away_fgm_39'][h_games[j]] + 3*game_data['away_fgm_40_49'][h_games[j]] 
                        + 3*game_data['away_fgm_50'][h_games[j]]) / (game_data['away_rush_plays'][h_games[j]] 
                                                                     + game_data['away_pass_att'][h_games[j]] 
                                                                     + game_data['away_sacks_taken'][h_games[j]])
        
        a_home_pts_j = (6*game_data['home_rush_tds'][a_games[j]] + 6*game_data['home_pass_tds'][a_games[j]] 
                        + game_data['home_pat_m'][a_games[j]] + 2*game_data['home_2pt_conv_suc'][a_games[j]] 
                        + 3*game_data['home_fgm_39'][a_games[j]] + 3*game_data['home_fgm_40_49'][a_games[j]] 
                        + 3*game_data['home_fgm_50'][a_games[j]]) / (game_data['home_rush_plays'][a_games[j]] 
                                                                     + game_data['home_pass_att'][a_games[j]] 
                                                                     + game_data['home_sacks_taken'][a_games[j]])
        
        a_away_pts_j = (6*game_data['away_rush_tds'][a_games[j]] + 6*game_data['away_pass_tds'][a_games[j]] 
                        + game_data['away_pat_m'][a_games[j]] + 2*game_data['away_2pt_conv_suc'][a_games[j]] 
                        + 3*game_data['away_fgm_39'][a_games[j]] + 3*game_data['away_fgm_40_49'][a_games[j]] 
                        + 3*game_data['away_fgm_50'][a_games[j]]) / (game_data['away_rush_plays'][a_games[j]] 
                                                                     + game_data['away_pass_att'][a_games[j]] 
                                                                     + game_data['away_sacks_taken'][a_games[j]])
        
        if(game_data['home_team_code'][h_games[j]] == h_team):
            home_off_pts_per_play[i] += past_game_coef[j]*match_visit_stat_coef*h_home_pts_j
            home_def_pts_per_play[i] += past_game_coef[j]*match_visit_stat_coef*h_away_pts_j
            h_divisor += past_game_coef[j]*match_visit_stat_coef
        else:
            home_off_pts_per_play[i] += past_game_coef[j]*non_match_visit_stat_coef*h_away_pts_j
            home_def_pts_per_play[i] += past_game_coef[j]*non_match_visit_stat_coef*h_home_pts_j
            h_divisor += past_game_coef[j]*non_match_visit_stat_coef
            
        if(game_data['home_team_code'][a_games[j]] == a_team):
            away_off_pts_per_play[i] += past_game_coef[j]*non_match_visit_stat_coef*a_home_pts_j
            away_def_pts_per_play[i] += past_game_coef[j]*non_match_visit_stat_coef*a_away_pts_j
            a_divisor += past_game_coef[j]*non_match_visit_stat_coef
        else:
            away_off_pts_per_play[i] += past_game_coef[j]*match_visit_stat_coef*a_away_pts_j
            away_def_pts_per_play[i] += past_game_coef[j]*match_visit_stat_coef*a_home_pts_j
            a_divisor += past_game_coef[j]*match_visit_stat_coef
    
    home_off_pts_per_play[i] = home_off_pts_per_play[i] / h_divisor
    away_off_pts_per_play[i] = away_off_pts_per_play[i] / a_divisor
    home_def_pts_per_play[i] = home_def_pts_per_play[i] / h_divisor
    away_def_pts_per_play[i] = away_def_pts_per_play[i] / a_divisor
    
    
    h_divisor = 0
    a_divisor = 0
    for j in range(len(past_game_coef)):
        h_home_yds_j = (game_data['home_rush_yds'][h_games[j]] + game_data['home_gross_pass_yds'][h_games[j]] 
                        - game_data['home_sack_yds_taken'][h_games[j]]) / (game_data['home_rush_plays'][h_games[j]] 
                                                                           + game_data['home_pass_att'][h_games[j]] 
                                                                           + game_data['home_sacks_taken'][h_games[j]])
        
        h_away_yds_j = (game_data['away_rush_yds'][h_games[j]] + game_data['away_gross_pass_yds'][h_games[j]] 
                        - game_data['away_sack_yds_taken'][h_games[j]]) / (game_data['away_rush_plays'][h_games[j]] 
                                                                           + game_data['away_pass_att'][h_games[j]] 
                                                                           + game_data['away_sacks_taken'][h_games[j]])
        
        a_home_yds_j = (game_data['home_rush_yds'][a_games[j]] + game_data['home_gross_pass_yds'][a_games[j]] 
                        - game_data['home_sack_yds_taken'][a_games[j]]) / (game_data['home_rush_plays'][a_games[j]] 
                                                                           + game_data['home_pass_att'][a_games[j]] 
                                                                           + game_data['home_sacks_taken'][a_games[j]])
        
        a_away_yds_j = (game_data['away_rush_yds'][a_games[j]] + game_data['away_gross_pass_yds'][a_games[j]] 
                        - game_data['away_sack_yds_taken'][a_games[j]]) / (game_data['away_rush_plays'][a_games[j]] 
                                                                           + game_data['away_pass_att'][a_games[j]] 
                                                                           + game_data['away_sacks_taken'][a_games[j]])
        
        if(game_data['home_team_code'][h_games[j]] == h_team):
            home_off_yds_per_play[i] += past_game_coef[j]*match_visit_stat_coef*h_home_yds_j
            home_def_yds_per_play[i] += past_game_coef[j]*match_visit_stat_coef*h_away_yds_j
            h_divisor += past_game_coef[j]*match_visit_stat_coef
        else:
            home_off_yds_per_play[i] += past_game_coef[j]*non_match_visit_stat_coef*h_away_yds_j
            home_def_yds_per_play[i] += past_game_coef[j]*non_match_visit_stat_coef*h_home_yds_j
            h_divisor += past_game_coef[j]*non_match_visit_stat_coef
            
        if(game_data['home_team_code'][a_games[j]] == a_team):
            away_off_yds_per_play[i] += past_game_coef[j]*non_match_visit_stat_coef*a_home_yds_j
            away_def_yds_per_play[i] += past_game_coef[j]*non_match_visit_stat_coef*a_away_yds_j
            a_divisor += past_game_coef[j]*non_match_visit_stat_coef
        else:
            away_off_yds_per_play[i] += past_game_coef[j]*match_visit_stat_coef*a_away_yds_j
            away_def_yds_per_play[i] += past_game_coef[j]*match_visit_stat_coef*a_home_yds_j
            a_divisor += past_game_coef[j]*match_visit_stat_coef
    
    home_off_yds_per_play[i] = home_off_yds_per_play[i] / h_divisor
    away_off_yds_per_play[i] = away_off_yds_per_play[i] / a_divisor
    home_def_yds_per_play[i] = home_def_yds_per_play[i] / h_divisor
    away_def_yds_per_play[i] = away_def_yds_per_play[i] / a_divisor
    
    
    h_divisor = 0
    a_divisor = 0
    for j in range(len(past_game_coef)):
        h_home_time_j = game_data['home_total_pos_time'][h_games[j]] / (game_data['home_rush_plays'][h_games[j]] 
                                                                        + game_data['home_pass_att'][h_games[j]] 
                                                                        + game_data['home_sacks_taken'][h_games[j]]
                                                                        + game_data['home_punt_returns'][h_games[j]] 
                                                                        + game_data['home_kickoff_returns'][h_games[j]])
        
        h_away_time_j = game_data['away_total_pos_time'][h_games[j]] / (game_data['away_rush_plays'][h_games[j]] 
                                                                        + game_data['away_pass_att'][h_games[j]] 
                                                                        + game_data['away_sacks_taken'][h_games[j]]
                                                                        + game_data['away_punt_returns'][h_games[j]] 
                                                                        + game_data['away_kickoff_returns'][h_games[j]])
        
        a_home_time_j = game_data['home_total_pos_time'][a_games[j]] / (game_data['home_rush_plays'][a_games[j]] 
                                                                        + game_data['home_pass_att'][a_games[j]] 
                                                                        + game_data['home_sacks_taken'][a_games[j]]
                                                                        + game_data['home_punt_returns'][a_games[j]] 
                                                                        + game_data['home_kickoff_returns'][a_games[j]])
        
        a_away_time_j = game_data['away_total_pos_time'][a_games[j]] / (game_data['away_rush_plays'][a_games[j]] 
                                                                        + game_data['away_pass_att'][a_games[j]] 
                                                                        + game_data['away_sacks_taken'][a_games[j]]
                                                                        + game_data['away_punt_returns'][a_games[j]] 
                                                                        + game_data['away_kickoff_returns'][a_games[j]])
        
        if(game_data['home_team_code'][h_games[j]] == h_team):
            home_off_time_per_play[i] += past_game_coef[j]*match_visit_stat_coef*h_home_time_j
            home_def_time_per_play[i] += past_game_coef[j]*match_visit_stat_coef*h_away_time_j
            h_divisor += past_game_coef[j]*match_visit_stat_coef
        else:
            home_off_time_per_play[i] += past_game_coef[j]*non_match_visit_stat_coef*h_away_time_j
            home_def_time_per_play[i] += past_game_coef[j]*non_match_visit_stat_coef*h_home_time_j
            h_divisor += past_game_coef[j]*non_match_visit_stat_coef
            
        if(game_data['home_team_code'][a_games[j]] == a_team):
            away_off_time_per_play[i] += past_game_coef[j]*non_match_visit_stat_coef*a_home_time_j
            away_def_time_per_play[i] += past_game_coef[j]*non_match_visit_stat_coef*a_away_time_j
            a_divisor += past_game_coef[j]*non_match_visit_stat_coef
        else:
            away_off_time_per_play[i] += past_game_coef[j]*match_visit_stat_coef*a_away_time_j
            away_def_time_per_play[i] += past_game_coef[j]*match_visit_stat_coef*a_home_time_j
            a_divisor += past_game_coef[j]*match_visit_stat_coef
    
    home_off_time_per_play[i] = home_off_time_per_play[i] / h_divisor
    away_off_time_per_play[i] = away_off_time_per_play[i] / a_divisor
    home_def_time_per_play[i] = home_def_time_per_play[i] / h_divisor
    away_def_time_per_play[i] = away_def_time_per_play[i] / a_divisor
    
    
    h_divisor = 0
    a_divisor = 0
    for j in range(len(past_game_coef)):
        h_home_perc_j = game_data['home_rush_plays'][h_games[j]] / (game_data['home_rush_plays'][h_games[j]] 
                                                                    + game_data['home_pass_att'][h_games[j]] 
                                                                    + game_data['home_sacks_taken'][h_games[j]])
        
        h_away_perc_j = game_data['away_rush_plays'][h_games[j]] / (game_data['away_rush_plays'][h_games[j]] 
                                                                    + game_data['away_pass_att'][h_games[j]] 
                                                                    + game_data['away_sacks_taken'][h_games[j]])
        
        a_home_perc_j = game_data['home_rush_plays'][a_games[j]] / (game_data['home_rush_plays'][a_games[j]] 
                                                                    + game_data['home_pass_att'][a_games[j]] 
                                                                    + game_data['home_sacks_taken'][a_games[j]])
        
        a_away_perc_j = game_data['away_rush_plays'][a_games[j]] / (game_data['away_rush_plays'][a_games[j]] 
                                                                    + game_data['away_pass_att'][a_games[j]] 
                                                                    + game_data['away_sacks_taken'][a_games[j]])
        
        if(game_data['home_team_code'][h_games[j]] == h_team):
            home_off_perc_rush[i] += past_game_coef[j]*match_visit_stat_coef*h_home_perc_j
            home_def_perc_rush[i] += past_game_coef[j]*match_visit_stat_coef*h_away_perc_j
            h_divisor += past_game_coef[j]*match_visit_stat_coef
        else:
            home_off_perc_rush[i] += past_game_coef[j]*non_match_visit_stat_coef*h_away_perc_j
            home_def_perc_rush[i] += past_game_coef[j]*non_match_visit_stat_coef*h_home_perc_j
            h_divisor += past_game_coef[j]*non_match_visit_stat_coef
            
        if(game_data['home_team_code'][a_games[j]] == a_team):
            away_off_perc_rush[i] += past_game_coef[j]*non_match_visit_stat_coef*a_home_perc_j
            away_def_perc_rush[i] += past_game_coef[j]*non_match_visit_stat_coef*a_away_perc_j
            a_divisor += past_game_coef[j]*non_match_visit_stat_coef
        else:
            away_off_perc_rush[i] += past_game_coef[j]*match_visit_stat_coef*a_away_perc_j
            away_def_perc_rush[i] += past_game_coef[j]*match_visit_stat_coef*a_home_perc_j
            a_divisor += past_game_coef[j]*match_visit_stat_coef
    
    home_off_perc_rush[i] = home_off_perc_rush[i] / h_divisor
    away_off_perc_rush[i] = away_off_perc_rush[i] / a_divisor
    home_def_perc_rush[i] = home_def_perc_rush[i] / h_divisor
    away_def_perc_rush[i] = away_def_perc_rush[i] / a_divisor
    
    
    h_divisor = 0
    a_divisor = 0
    for j in range(len(past_game_coef)):
        h_home_perc_j = game_data['home_rush_first_downs'][h_games[j]] / (game_data['home_rush_first_downs'][h_games[j]] 
                                                                          + game_data['home_pass_first_downs'][h_games[j]])
        
        h_away_perc_j = game_data['away_rush_first_downs'][h_games[j]] / (game_data['away_rush_first_downs'][h_games[j]] 
                                                                          + game_data['away_pass_first_downs'][h_games[j]])
        
        a_home_perc_j = game_data['home_rush_first_downs'][a_games[j]] / (game_data['home_rush_first_downs'][a_games[j]] 
                                                                          + game_data['home_pass_first_downs'][a_games[j]])
        
        a_away_perc_j = game_data['away_rush_first_downs'][a_games[j]] / (game_data['away_rush_first_downs'][a_games[j]] 
                                                                          + game_data['away_pass_first_downs'][a_games[j]])
        
        if(game_data['home_team_code'][h_games[j]] == h_team):
            home_off_perc_fstdown_rush[i] += past_game_coef[j]*match_visit_stat_coef*h_home_perc_j
            home_def_perc_fstdown_rush[i] += past_game_coef[j]*match_visit_stat_coef*h_away_perc_j
            h_divisor += past_game_coef[j]*match_visit_stat_coef
        else:
            home_off_perc_fstdown_rush[i] += past_game_coef[j]*non_match_visit_stat_coef*h_away_perc_j
            home_def_perc_fstdown_rush[i] += past_game_coef[j]*non_match_visit_stat_coef*h_home_perc_j
            h_divisor += past_game_coef[j]*non_match_visit_stat_coef
            
        if(game_data['home_team_code'][a_games[j]] == a_team):
            away_off_perc_fstdown_rush[i] += past_game_coef[j]*non_match_visit_stat_coef*a_home_perc_j
            away_def_perc_fstdown_rush[i] += past_game_coef[j]*non_match_visit_stat_coef*a_away_perc_j
            a_divisor += past_game_coef[j]*non_match_visit_stat_coef
        else:
            away_off_perc_fstdown_rush[i] += past_game_coef[j]*match_visit_stat_coef*a_away_perc_j
            away_def_perc_fstdown_rush[i] += past_game_coef[j]*match_visit_stat_coef*a_home_perc_j
            a_divisor += past_game_coef[j]*match_visit_stat_coef
    
    home_off_perc_fstdown_rush[i] = home_off_perc_fstdown_rush[i] / h_divisor
    away_off_perc_fstdown_rush[i] = away_off_perc_fstdown_rush[i] / a_divisor
    home_def_perc_fstdown_rush[i] = home_def_perc_fstdown_rush[i] / h_divisor
    away_def_perc_fstdown_rush[i] = away_def_perc_fstdown_rush[i] / a_divisor
    
    
    h_divisor = 0
    a_divisor = 0
    for j in range(len(past_game_coef)):
        h_home_epr_j = game_data['home_explosive_plays'][h_games[j]] / (game_data['home_rush_plays'][h_games[j]] 
                                                                        + game_data['home_pass_att'][h_games[j]] 
                                                                        + game_data['home_sacks_taken'][h_games[j]])
        
        h_away_epr_j = game_data['away_explosive_plays'][h_games[j]] / (game_data['away_rush_plays'][h_games[j]] 
                                                                        + game_data['away_pass_att'][h_games[j]] 
                                                                        + game_data['away_sacks_taken'][h_games[j]])
        
        a_home_epr_j = game_data['home_explosive_plays'][a_games[j]] / (game_data['home_rush_plays'][a_games[j]] 
                                                                        + game_data['home_pass_att'][a_games[j]] 
                                                                        + game_data['home_sacks_taken'][a_games[j]])
        
        a_away_epr_j = game_data['away_explosive_plays'][a_games[j]] / (game_data['away_rush_plays'][a_games[j]] 
                                                                        + game_data['away_pass_att'][a_games[j]] 
                                                                        + game_data['away_sacks_taken'][a_games[j]])
        
        if(game_data['home_team_code'][h_games[j]] == h_team):
            home_off_epr[i] += past_game_coef[j]*match_visit_stat_coef*h_home_epr_j
            home_def_epr[i] += past_game_coef[j]*match_visit_stat_coef*h_away_epr_j
            h_divisor += past_game_coef[j]*match_visit_stat_coef
        else:
            home_off_epr[i] += past_game_coef[j]*non_match_visit_stat_coef*h_away_epr_j
            home_def_epr[i] += past_game_coef[j]*non_match_visit_stat_coef*h_home_epr_j
            h_divisor += past_game_coef[j]*non_match_visit_stat_coef
            
        if(game_data['home_team_code'][a_games[j]] == a_team):
            away_off_epr[i] += past_game_coef[j]*non_match_visit_stat_coef*a_home_epr_j
            away_def_epr[i] += past_game_coef[j]*non_match_visit_stat_coef*a_away_epr_j
            a_divisor += past_game_coef[j]*non_match_visit_stat_coef
        else:
            away_off_epr[i] += past_game_coef[j]*match_visit_stat_coef*a_away_epr_j
            away_def_epr[i] += past_game_coef[j]*match_visit_stat_coef*a_home_epr_j
            a_divisor += past_game_coef[j]*match_visit_stat_coef
    
    home_off_epr[i] = home_off_epr[i] / h_divisor
    away_off_epr[i] = away_off_epr[i] / a_divisor
    home_def_epr[i] = home_def_epr[i] / h_divisor
    away_def_epr[i] = away_def_epr[i] / a_divisor
    
    
    h_divisor = 0
    a_divisor = 0
    for j in range(len(past_game_coef)):
        h_home_fumble_j = game_data['home_fumbles_lost'][h_games[j]] / (game_data['home_rush_plays'][h_games[j]] 
                                                                        + game_data['home_pass_att'][h_games[j]] 
                                                                        + game_data['home_sacks_taken'][h_games[j]] 
                                                                        + game_data['home_punt_returns'][h_games[j]] 
                                                                        + game_data['home_kickoff_returns'][h_games[j]])
        
        h_away_fumble_j = game_data['away_fumbles_lost'][h_games[j]] / (game_data['away_rush_plays'][h_games[j]] 
                                                                        + game_data['away_pass_att'][h_games[j]] 
                                                                        + game_data['away_sacks_taken'][h_games[j]] 
                                                                        + game_data['away_punt_returns'][h_games[j]] 
                                                                        + game_data['away_kickoff_returns'][h_games[j]])
        
        a_home_fumble_j = game_data['home_fumbles_lost'][a_games[j]] / (game_data['home_rush_plays'][a_games[j]] 
                                                                        + game_data['home_pass_att'][a_games[j]] 
                                                                        + game_data['home_sacks_taken'][a_games[j]] 
                                                                        + game_data['home_punt_returns'][a_games[j]] 
                                                                        + game_data['home_kickoff_returns'][a_games[j]])
        
        a_away_fumble_j = game_data['away_fumbles_lost'][a_games[j]] / (game_data['away_rush_plays'][a_games[j]] 
                                                                        + game_data['away_pass_att'][a_games[j]] 
                                                                        + game_data['away_sacks_taken'][a_games[j]] 
                                                                        + game_data['away_punt_returns'][a_games[j]] 
                                                                        + game_data['away_kickoff_returns'][a_games[j]])
        
        if(game_data['home_team_code'][h_games[j]] == h_team):
            home_off_fumble_rate[i] += past_game_coef[j]*match_visit_stat_coef*h_home_fumble_j
            home_def_fumble_rate[i] += past_game_coef[j]*match_visit_stat_coef*h_away_fumble_j
            h_divisor += past_game_coef[j]*match_visit_stat_coef
        else:
            home_off_fumble_rate[i] += past_game_coef[j]*non_match_visit_stat_coef*h_away_fumble_j
            home_def_fumble_rate[i] += past_game_coef[j]*non_match_visit_stat_coef*h_home_fumble_j
            h_divisor += past_game_coef[j]*non_match_visit_stat_coef
            
        if(game_data['home_team_code'][a_games[j]] == a_team):
            away_off_fumble_rate[i] += past_game_coef[j]*non_match_visit_stat_coef*a_home_fumble_j
            away_def_fumble_rate[i] += past_game_coef[j]*non_match_visit_stat_coef*a_away_fumble_j
            a_divisor += past_game_coef[j]*non_match_visit_stat_coef
        else:
            away_off_fumble_rate[i] += past_game_coef[j]*match_visit_stat_coef*a_away_fumble_j
            away_def_fumble_rate[i] += past_game_coef[j]*match_visit_stat_coef*a_home_fumble_j
            a_divisor += past_game_coef[j]*match_visit_stat_coef
    
    home_off_fumble_rate[i] = home_off_fumble_rate[i] / h_divisor
    away_off_fumble_rate[i] = away_off_fumble_rate[i] / a_divisor
    home_def_fumble_rate[i] = home_def_fumble_rate[i] / h_divisor
    away_def_fumble_rate[i] = away_def_fumble_rate[i] / a_divisor
    
    
    h_divisor = 0
    a_divisor = 0
    for j in range(len(past_game_coef)):
        h_home_int_j = game_data['home_ints_thrown'][h_games[j]] / game_data['home_pass_att'][h_games[j]]
        h_away_int_j = game_data['away_ints_thrown'][h_games[j]] / game_data['away_pass_att'][h_games[j]]
        a_home_int_j = game_data['home_ints_thrown'][a_games[j]] / game_data['home_pass_att'][a_games[j]]
        a_away_int_j = game_data['away_ints_thrown'][a_games[j]] / game_data['away_pass_att'][a_games[j]]
                    
        if(game_data['home_team_code'][h_games[j]] == h_team):
            home_off_int_rate[i] += past_game_coef[j]*match_visit_stat_coef*h_home_int_j
            home_def_int_rate[i] += past_game_coef[j]*match_visit_stat_coef*h_away_int_j
            h_divisor += past_game_coef[j]*match_visit_stat_coef
        else:
            home_off_int_rate[i] += past_game_coef[j]*non_match_visit_stat_coef*h_away_int_j
            home_def_int_rate[i] += past_game_coef[j]*non_match_visit_stat_coef*h_home_int_j
            h_divisor += past_game_coef[j]*non_match_visit_stat_coef
            
        if(game_data['home_team_code'][a_games[j]] == a_team):
            away_off_int_rate[i] += past_game_coef[j]*non_match_visit_stat_coef*a_home_int_j
            away_def_int_rate[i] += past_game_coef[j]*non_match_visit_stat_coef*a_away_int_j
            a_divisor += past_game_coef[j]*non_match_visit_stat_coef
        else:
            away_off_int_rate[i] += past_game_coef[j]*match_visit_stat_coef*a_away_int_j
            away_def_int_rate[i] += past_game_coef[j]*match_visit_stat_coef*a_home_int_j
            a_divisor += past_game_coef[j]*match_visit_stat_coef
    
    home_off_int_rate[i] = home_off_int_rate[i] / h_divisor
    away_off_int_rate[i] = away_off_int_rate[i] / a_divisor
    home_def_int_rate[i] = home_def_int_rate[i] / h_divisor
    away_def_int_rate[i] = away_def_int_rate[i] / a_divisor
    
    
    h_divisor = 0
    a_divisor = 0
    for j in range(len(past_game_coef)):
        h_home_sack_j = game_data['home_sacks_taken'][h_games[j]] / (game_data['home_pass_att'][h_games[j]] 
                                                                     + game_data['home_sacks_taken'][h_games[j]])
        h_away_sack_j = game_data['away_sacks_taken'][h_games[j]] / (game_data['away_pass_att'][h_games[j]] 
                                                                     + game_data['away_sacks_taken'][h_games[j]])
        a_home_sack_j = game_data['home_sacks_taken'][a_games[j]] / (game_data['home_pass_att'][a_games[j]] 
                                                                     + game_data['home_sacks_taken'][a_games[j]])
        a_away_sack_j = game_data['away_sacks_taken'][a_games[j]] / (game_data['away_pass_att'][a_games[j]] 
                                                                     + game_data['away_sacks_taken'][a_games[j]])
                    
        if(game_data['home_team_code'][h_games[j]] == h_team):
            home_off_sack_rate[i] += past_game_coef[j]*match_visit_stat_coef*h_home_sack_j
            home_def_sack_rate[i] += past_game_coef[j]*match_visit_stat_coef*h_away_sack_j
            h_divisor += past_game_coef[j]*match_visit_stat_coef
        else:
            home_off_sack_rate[i] += past_game_coef[j]*non_match_visit_stat_coef*h_away_sack_j
            home_def_sack_rate[i] += past_game_coef[j]*non_match_visit_stat_coef*h_home_sack_j
            h_divisor += past_game_coef[j]*non_match_visit_stat_coef
            
        if(game_data['home_team_code'][a_games[j]] == a_team):
            away_off_sack_rate[i] += past_game_coef[j]*non_match_visit_stat_coef*a_home_sack_j
            away_def_sack_rate[i] += past_game_coef[j]*non_match_visit_stat_coef*a_away_sack_j
            a_divisor += past_game_coef[j]*non_match_visit_stat_coef
        else:
            away_off_sack_rate[i] += past_game_coef[j]*match_visit_stat_coef*a_away_sack_j
            away_def_sack_rate[i] += past_game_coef[j]*match_visit_stat_coef*a_home_sack_j
            a_divisor += past_game_coef[j]*match_visit_stat_coef
    
    home_off_sack_rate[i] = home_off_sack_rate[i] / h_divisor
    away_off_sack_rate[i] = away_off_sack_rate[i] / a_divisor
    home_def_sack_rate[i] = home_def_sack_rate[i] / h_divisor
    away_def_sack_rate[i] = away_def_sack_rate[i] / a_divisor
    
    
    h_divisor = 0
    a_divisor = 0
    for j in range(len(past_game_coef)):
        h_home_tom_j = game_data['away_turnovers'][h_games[j]] - game_data['home_turnovers'][h_games[j]]
        h_away_tom_j = game_data['home_turnovers'][h_games[j]] - game_data['away_turnovers'][h_games[j]]
        a_home_tom_j = game_data['away_turnovers'][a_games[j]] - game_data['home_turnovers'][a_games[j]]
        a_away_tom_j = game_data['home_turnovers'][a_games[j]] - game_data['away_turnovers'][a_games[j]]
        
        if(game_data['home_team_code'][h_games[j]] == h_team):
            home_turnover_margin[i] += past_game_coef[j]*match_visit_stat_coef*h_home_tom_j
            h_divisor += past_game_coef[j]*match_visit_stat_coef
        else:
            home_turnover_margin[i] += past_game_coef[j]*non_match_visit_stat_coef*h_away_tom_j
            h_divisor += past_game_coef[j]*non_match_visit_stat_coef
            
        if(game_data['home_team_code'][a_games[j]] == a_team):
            away_turnover_margin[i] += past_game_coef[j]*non_match_visit_stat_coef*a_home_tom_j
            a_divisor += past_game_coef[j]*non_match_visit_stat_coef
        else:
            away_turnover_margin[i] += past_game_coef[j]*match_visit_stat_coef*a_away_int_j
            a_divisor += past_game_coef[j]*match_visit_stat_coef
    
    home_turnover_margin[i] = home_turnover_margin[i] / h_divisor
    away_turnover_margin[i] = away_turnover_margin[i] / a_divisor
    
    
    previous_games = game_data[:start_game_index + i]
    h_last_away = previous_games.index[previous_games['away_team_code']==h_team].tolist()[-1]
    h_last_home = previous_games.index[previous_games['home_team_code']==h_team].tolist()[-1]
    a_last_away = previous_games.index[previous_games['away_team_code']==a_team].tolist()[-1]
    a_last_home = previous_games.index[previous_games['home_team_code']==a_team].tolist()[-1]
    
    if(h_last_away > h_last_home):
        home_lw_turnover_margin[i] = game_data['home_turnovers'][h_last_away] - game_data['away_turnovers'][h_last_away]
    else:
        home_lw_turnover_margin[i] = game_data['away_turnovers'][h_last_home] - game_data['home_turnovers'][h_last_home]
        
    if(a_last_away > a_last_home):
        away_lw_turnover_margin[i] = game_data['home_turnovers'][a_last_away] - game_data['away_turnovers'][a_last_away]
    else:
        away_lw_turnover_margin[i] = game_data['away_turnovers'][a_last_home] - game_data['home_turnovers'][a_last_home]
        
    
    h_divisor = 0
    a_divisor = 0
    for j in range(len(past_game_coef)):
        h_home_tdcr_j = game_data['home_third_down_suc'][h_games[j]] / game_data['home_third_down_att'][h_games[j]]
        h_away_tdcr_j = game_data['away_third_down_suc'][h_games[j]] / game_data['away_third_down_att'][h_games[j]]
        a_home_tdcr_j = game_data['home_third_down_suc'][a_games[j]] / game_data['home_third_down_att'][a_games[j]]
        a_away_tdcr_j = game_data['away_third_down_suc'][a_games[j]] / game_data['away_third_down_att'][a_games[j]]
        
        if(game_data['home_team_code'][h_games[j]] == h_team):
            home_off_thrdown_conv[i] += past_game_coef[j]*match_visit_stat_coef*h_home_tdcr_j
            home_def_thrdown_conv[i] += past_game_coef[j]*match_visit_stat_coef*h_away_tdcr_j
            h_divisor += past_game_coef[j]*match_visit_stat_coef
        else:
            home_off_thrdown_conv[i] += past_game_coef[j]*non_match_visit_stat_coef*h_away_tdcr_j
            home_def_thrdown_conv[i] += past_game_coef[j]*non_match_visit_stat_coef*h_home_tdcr_j
            h_divisor += past_game_coef[j]*non_match_visit_stat_coef
            
        if(game_data['home_team_code'][a_games[j]] == a_team):
            away_off_thrdown_conv[i] += past_game_coef[j]*non_match_visit_stat_coef*a_home_tdcr_j
            away_def_thrdown_conv[i] += past_game_coef[j]*non_match_visit_stat_coef*a_away_tdcr_j
            a_divisor += past_game_coef[j]*non_match_visit_stat_coef
        else:
            away_off_thrdown_conv[i] += past_game_coef[j]*match_visit_stat_coef*a_away_tdcr_j
            away_def_thrdown_conv[i] += past_game_coef[j]*match_visit_stat_coef*a_home_tdcr_j
            a_divisor += past_game_coef[j]*match_visit_stat_coef
    
    home_off_thrdown_conv[i] = home_off_thrdown_conv[i] / h_divisor
    away_off_thrdown_conv[i] = away_off_thrdown_conv[i] / a_divisor
    home_def_thrdown_conv[i] = home_def_thrdown_conv[i] / h_divisor
    away_def_thrdown_conv[i] = away_def_thrdown_conv[i] / a_divisor
    
    
    h_divisor = 0
    a_divisor = 0
    for j in range(len(past_game_coef)):
        h_home_stat_j = game_data['away_pts'][h_games[j]] / ((game_data['away_rush_yds'][h_games[j]] 
                                                             + game_data['away_gross_pass_yds'][h_games[j]] 
                                                             - game_data['away_sack_yds_taken'][h_games[j]]) / 100)
        
        h_away_stat_j = game_data['home_pts'][h_games[j]] / ((game_data['home_rush_yds'][h_games[j]] 
                                                             + game_data['home_gross_pass_yds'][h_games[j]] 
                                                             - game_data['home_sack_yds_taken'][h_games[j]]) / 100)
        
        a_home_stat_j = game_data['away_pts'][a_games[j]] / ((game_data['away_rush_yds'][a_games[j]] 
                                                             + game_data['away_gross_pass_yds'][a_games[j]] 
                                                             - game_data['away_sack_yds_taken'][a_games[j]]) / 100)
        
        a_away_stat_j = game_data['home_pts'][a_games[j]] / ((game_data['home_rush_yds'][a_games[j]] 
                                                             + game_data['home_gross_pass_yds'][a_games[j]] 
                                                             - game_data['home_sack_yds_taken'][a_games[j]]) / 100)
        
        if(game_data['home_team_code'][h_games[j]] == h_team):
            home_def_pts_100[i] += past_game_coef[j]*match_visit_stat_coef*h_home_stat_j
            h_divisor += past_game_coef[j]*match_visit_stat_coef
        else:
            home_def_pts_100[i] += past_game_coef[j]*non_match_visit_stat_coef*h_away_stat_j
            h_divisor += past_game_coef[j]*non_match_visit_stat_coef
            
        if(game_data['home_team_code'][a_games[j]] == a_team):
            away_def_pts_100[i] += past_game_coef[j]*non_match_visit_stat_coef*a_home_stat_j
            a_divisor += past_game_coef[j]*non_match_visit_stat_coef
        else:
            away_def_pts_100[i] += past_game_coef[j]*match_visit_stat_coef*a_away_stat_j
            a_divisor += past_game_coef[j]*match_visit_stat_coef
    
    home_def_pts_100[i] = home_def_pts_100[i] / h_divisor
    away_def_pts_100[i] = away_def_pts_100[i] / a_divisor
    
    
    h_divisor = 0
    a_divisor = 0
    for j in range(len(past_game_coef)):
        h_home_stat_j = game_data['home_avg_sfp'][h_games[j]]
        h_away_stat_j = game_data['away_avg_sfp'][h_games[j]]
        a_home_stat_j = game_data['home_avg_sfp'][a_games[j]]
        a_away_stat_j = game_data['away_avg_sfp'][a_games[j]]
        
        if(game_data['home_team_code'][h_games[j]] == h_team):
            home_off_avg_starting_field_pos[i] += past_game_coef[j]*match_visit_stat_coef*h_home_stat_j
            home_def_avg_starting_field_pos[i] += past_game_coef[j]*match_visit_stat_coef*h_away_stat_j
            h_divisor += past_game_coef[j]*match_visit_stat_coef
        else:
            home_off_avg_starting_field_pos[i] += past_game_coef[j]*non_match_visit_stat_coef*h_away_stat_j
            home_def_avg_starting_field_pos[i] += past_game_coef[j]*non_match_visit_stat_coef*h_home_stat_j
            h_divisor += past_game_coef[j]*non_match_visit_stat_coef
            
        if(game_data['home_team_code'][a_games[j]] == a_team):
            away_off_avg_starting_field_pos[i] += past_game_coef[j]*non_match_visit_stat_coef*a_home_stat_j
            away_def_avg_starting_field_pos[i] += past_game_coef[j]*non_match_visit_stat_coef*a_away_stat_j
            a_divisor += past_game_coef[j]*non_match_visit_stat_coef
        else:
            away_off_avg_starting_field_pos[i] += past_game_coef[j]*match_visit_stat_coef*a_away_stat_j
            away_def_avg_starting_field_pos[i] += past_game_coef[j]*match_visit_stat_coef*a_home_stat_j
            a_divisor += past_game_coef[j]*match_visit_stat_coef
    
    home_off_avg_starting_field_pos[i] = home_off_avg_starting_field_pos[i] / h_divisor
    away_off_avg_starting_field_pos[i] = away_off_avg_starting_field_pos[i] / a_divisor
    home_def_avg_starting_field_pos[i] = home_def_avg_starting_field_pos[i] / h_divisor
    away_def_avg_starting_field_pos[i] = away_def_avg_starting_field_pos[i] / a_divisor
    
    
    h_divisor = 0
    a_divisor = 0
    for j in range(len(past_game_coef)):
        h_home_off_j = game_data['home_off_pen_yds'][h_games[j]]
        h_home_def_j = game_data['home_def_pen_yds'][h_games[j]]
        h_away_off_j = game_data['away_off_pen_yds'][h_games[j]]
        h_away_def_j = game_data['away_def_pen_yds'][h_games[j]]
        a_home_off_j = game_data['home_off_pen_yds'][a_games[j]]
        a_home_def_j = game_data['home_def_pen_yds'][a_games[j]]
        a_away_off_j = game_data['away_off_pen_yds'][a_games[j]]
        a_away_def_j = game_data['away_def_pen_yds'][a_games[j]]
        
        if(game_data['home_team_code'][h_games[j]] == h_team):
            home_off_pen_yds[i] += past_game_coef[j]*match_visit_stat_coef*h_home_off_j
            home_def_pen_yds[i] += past_game_coef[j]*match_visit_stat_coef*h_home_def_j
            home_opp_off_pen_yds[i] += past_game_coef[j]*match_visit_stat_coef*h_away_off_j
            home_opp_def_pen_yds[i] += past_game_coef[j]*match_visit_stat_coef*h_away_def_j
            h_divisor += past_game_coef[j]*match_visit_stat_coef
        else:
            home_off_pen_yds[i] += past_game_coef[j]*non_match_visit_stat_coef*h_away_off_j
            home_def_pen_yds[i] += past_game_coef[j]*non_match_visit_stat_coef*h_away_def_j
            home_opp_off_pen_yds[i] += past_game_coef[j]*non_match_visit_stat_coef*h_home_off_j
            home_opp_def_pen_yds[i] += past_game_coef[j]*non_match_visit_stat_coef*h_home_def_j
            h_divisor += past_game_coef[j]*non_match_visit_stat_coef
            
        if(game_data['home_team_code'][a_games[j]] == a_team):
            away_off_pen_yds[i] += past_game_coef[j]*non_match_visit_stat_coef*a_home_off_j
            away_def_pen_yds[i] += past_game_coef[j]*non_match_visit_stat_coef*a_home_def_j
            away_opp_off_pen_yds[i] += past_game_coef[j]*non_match_visit_stat_coef*a_away_off_j
            away_opp_def_pen_yds[i] += past_game_coef[j]*non_match_visit_stat_coef*a_away_def_j
            a_divisor += past_game_coef[j]*non_match_visit_stat_coef
        else:
            away_off_pen_yds[i] += past_game_coef[j]*match_visit_stat_coef*a_away_off_j
            away_def_pen_yds[i] += past_game_coef[j]*match_visit_stat_coef*a_away_def_j
            away_opp_off_pen_yds[i] += past_game_coef[j]*match_visit_stat_coef*a_home_off_j
            away_opp_def_pen_yds[i] += past_game_coef[j]*match_visit_stat_coef*a_home_def_j
            a_divisor += past_game_coef[j]*match_visit_stat_coef
    
    home_off_pen_yds[i] = home_off_pen_yds[i] / h_divisor
    away_off_pen_yds[i] = away_off_pen_yds[i] / a_divisor
    home_def_pen_yds[i] = home_def_pen_yds[i] / h_divisor
    away_def_pen_yds[i] = away_def_pen_yds[i] / a_divisor
    
    home_opp_off_pen_yds[i] = home_opp_off_pen_yds[i] / h_divisor
    away_opp_off_pen_yds[i] = away_opp_off_pen_yds[i] / a_divisor
    home_opp_def_pen_yds[i] = home_opp_def_pen_yds[i] / h_divisor
    away_opp_def_pen_yds[i] = away_opp_def_pen_yds[i] / a_divisor
    
    
    h_divisor = 0
    a_divisor = 0
    for j in range(len(past_game_coef)):
        h_home_stat_j = game_data['home_rush_yds'][h_games[j]] / game_data['home_rush_plays'][h_games[j]] 
        h_away_stat_j = game_data['away_rush_yds'][h_games[j]] / game_data['away_rush_plays'][h_games[j]] 
        a_home_stat_j = game_data['home_rush_yds'][a_games[j]] / game_data['home_rush_plays'][a_games[j]]
        a_away_stat_j = game_data['away_rush_yds'][a_games[j]] / game_data['away_rush_plays'][a_games[j]]
        
        if(game_data['home_team_code'][h_games[j]] == h_team):
            home_off_rush_ypc[i] += past_game_coef[j]*match_visit_stat_coef*h_home_stat_j
            home_def_rush_ypc[i] += past_game_coef[j]*match_visit_stat_coef*h_away_stat_j
            h_divisor += past_game_coef[j]*match_visit_stat_coef
        else:
            home_off_rush_ypc[i] += past_game_coef[j]*non_match_visit_stat_coef*h_away_stat_j
            home_def_rush_ypc[i] += past_game_coef[j]*non_match_visit_stat_coef*h_home_stat_j
            h_divisor += past_game_coef[j]*non_match_visit_stat_coef
            
        if(game_data['home_team_code'][a_games[j]] == a_team):
            away_off_rush_ypc[i] += past_game_coef[j]*non_match_visit_stat_coef*a_home_stat_j
            away_def_rush_ypc[i] += past_game_coef[j]*non_match_visit_stat_coef*a_away_stat_j
            a_divisor += past_game_coef[j]*non_match_visit_stat_coef
        else:
            away_off_rush_ypc[i] += past_game_coef[j]*match_visit_stat_coef*a_away_stat_j
            away_def_rush_ypc[i] += past_game_coef[j]*match_visit_stat_coef*a_home_stat_j
            a_divisor += past_game_coef[j]*match_visit_stat_coef
    
    home_off_rush_ypc[i] = home_off_rush_ypc[i] / h_divisor
    away_off_rush_ypc[i] = away_off_rush_ypc[i] / a_divisor
    home_def_rush_ypc[i] = home_def_rush_ypc[i] / h_divisor
    away_def_rush_ypc[i] = away_def_rush_ypc[i] / a_divisor

    
    h_divisor = 0
    a_divisor = 0
    for j in range(len(past_game_coef)):
        h_home_stat_j = game_data['home_rushes_ends'][h_games[j]] / game_data['home_rush_plays'][h_games[j]] 
        h_away_stat_j = game_data['away_rushes_ends'][h_games[j]] / game_data['away_rush_plays'][h_games[j]] 
        a_home_stat_j = game_data['home_rushes_ends'][a_games[j]] / game_data['home_rush_plays'][a_games[j]]
        a_away_stat_j = game_data['away_rushes_ends'][a_games[j]] / game_data['away_rush_plays'][a_games[j]]
        
        if(game_data['home_team_code'][h_games[j]] == h_team):
            home_off_perc_rush_ends[i] += past_game_coef[j]*match_visit_stat_coef*h_home_stat_j
            home_def_perc_rush_ends[i] += past_game_coef[j]*match_visit_stat_coef*h_away_stat_j
            h_divisor += past_game_coef[j]*match_visit_stat_coef
        else:
            home_off_perc_rush_ends[i] += past_game_coef[j]*non_match_visit_stat_coef*h_away_stat_j
            home_def_perc_rush_ends[i] += past_game_coef[j]*non_match_visit_stat_coef*h_home_stat_j
            h_divisor += past_game_coef[j]*non_match_visit_stat_coef
            
        if(game_data['home_team_code'][a_games[j]] == a_team):
            away_off_perc_rush_ends[i] += past_game_coef[j]*non_match_visit_stat_coef*a_home_stat_j
            away_def_perc_rush_ends[i] += past_game_coef[j]*non_match_visit_stat_coef*a_away_stat_j
            a_divisor += past_game_coef[j]*non_match_visit_stat_coef
        else:
            away_off_perc_rush_ends[i] += past_game_coef[j]*match_visit_stat_coef*a_away_stat_j
            away_def_perc_rush_ends[i] += past_game_coef[j]*match_visit_stat_coef*a_home_stat_j
            a_divisor += past_game_coef[j]*match_visit_stat_coef
    
    home_off_perc_rush_ends[i] = home_off_perc_rush_ends[i] / h_divisor
    away_off_perc_rush_ends[i] = away_off_perc_rush_ends[i] / a_divisor
    home_def_perc_rush_ends[i] = home_def_perc_rush_ends[i] / h_divisor
    away_def_perc_rush_ends[i] = away_def_perc_rush_ends[i] / a_divisor
    
    
    h_divisor = 0
    a_divisor = 0
    for j in range(len(past_game_coef)):
        h_home_stat_j = (game_data['home_early_down_rush_successes'][h_games[j]] 
                         / game_data['home_early_down_rush_att'][h_games[j]])
        h_away_stat_j = (game_data['away_early_down_rush_successes'][h_games[j]] 
                         / game_data['away_early_down_rush_att'][h_games[j]])
        a_home_stat_j = (game_data['home_early_down_rush_successes'][a_games[j]] 
                         / game_data['home_early_down_rush_att'][a_games[j]])
        a_away_stat_j = (game_data['away_early_down_rush_successes'][a_games[j]] 
                         / game_data['away_early_down_rush_att'][a_games[j]])
        
        if(game_data['home_team_code'][h_games[j]] == h_team):
            home_off_early_down_rush_suc[i] += past_game_coef[j]*match_visit_stat_coef*h_home_stat_j
            home_def_early_down_rush_suc[i] += past_game_coef[j]*match_visit_stat_coef*h_away_stat_j
            h_divisor += past_game_coef[j]*match_visit_stat_coef
        else:
            home_off_early_down_rush_suc[i] += past_game_coef[j]*non_match_visit_stat_coef*h_away_stat_j
            home_def_early_down_rush_suc[i] += past_game_coef[j]*non_match_visit_stat_coef*h_home_stat_j
            h_divisor += past_game_coef[j]*non_match_visit_stat_coef
            
        if(game_data['home_team_code'][a_games[j]] == a_team):
            away_off_early_down_rush_suc[i] += past_game_coef[j]*non_match_visit_stat_coef*a_home_stat_j
            away_def_early_down_rush_suc[i] += past_game_coef[j]*non_match_visit_stat_coef*a_away_stat_j
            a_divisor += past_game_coef[j]*non_match_visit_stat_coef
        else:
            away_off_early_down_rush_suc[i] += past_game_coef[j]*match_visit_stat_coef*a_away_stat_j
            away_def_early_down_rush_suc[i] += past_game_coef[j]*match_visit_stat_coef*a_home_stat_j
            a_divisor += past_game_coef[j]*match_visit_stat_coef
    
    home_off_early_down_rush_suc[i] = home_off_early_down_rush_suc[i] / h_divisor
    away_off_early_down_rush_suc[i] = away_off_early_down_rush_suc[i] / a_divisor
    home_def_early_down_rush_suc[i] = home_def_early_down_rush_suc[i] / h_divisor
    away_def_early_down_rush_suc[i] = away_def_early_down_rush_suc[i] / a_divisor
    
    
    h_divisor = 0
    a_divisor = 0
    for j in range(len(past_game_coef)):
        h_home_stat_j = (game_data['home_early_down_pass_successes'][h_games[j]] 
                         / game_data['home_early_down_pass_att'][h_games[j]])
        h_away_stat_j = (game_data['away_early_down_pass_successes'][h_games[j]] 
                         / game_data['away_early_down_pass_att'][h_games[j]]) 
        a_home_stat_j = (game_data['home_early_down_pass_successes'][a_games[j]] 
                         / game_data['home_early_down_pass_att'][a_games[j]])
        a_away_stat_j = (game_data['away_early_down_pass_successes'][a_games[j]] 
                         / game_data['away_early_down_pass_att'][a_games[j]])
        
        if(game_data['home_team_code'][h_games[j]] == h_team):
            home_off_early_down_pass_suc[i] += past_game_coef[j]*match_visit_stat_coef*h_home_stat_j
            home_def_early_down_pass_suc[i] += past_game_coef[j]*match_visit_stat_coef*h_away_stat_j
            h_divisor += past_game_coef[j]*match_visit_stat_coef
        else:
            home_off_early_down_pass_suc[i] += past_game_coef[j]*non_match_visit_stat_coef*h_away_stat_j
            home_def_early_down_pass_suc[i] += past_game_coef[j]*non_match_visit_stat_coef*h_home_stat_j
            h_divisor += past_game_coef[j]*non_match_visit_stat_coef
            
        if(game_data['home_team_code'][a_games[j]] == a_team):
            away_off_early_down_pass_suc[i] += past_game_coef[j]*non_match_visit_stat_coef*a_home_stat_j
            away_def_early_down_pass_suc[i] += past_game_coef[j]*non_match_visit_stat_coef*a_away_stat_j
            a_divisor += past_game_coef[j]*non_match_visit_stat_coef
        else:
            away_off_early_down_pass_suc[i] += past_game_coef[j]*match_visit_stat_coef*a_away_stat_j
            away_def_early_down_pass_suc[i] += past_game_coef[j]*match_visit_stat_coef*a_home_stat_j
            a_divisor += past_game_coef[j]*match_visit_stat_coef
    
    home_off_early_down_pass_suc[i] = home_off_early_down_pass_suc[i] / h_divisor
    away_off_early_down_pass_suc[i] = away_off_early_down_pass_suc[i] / a_divisor
    home_def_early_down_pass_suc[i] = home_def_early_down_pass_suc[i] / h_divisor
    away_def_early_down_pass_suc[i] = away_def_early_down_pass_suc[i] / a_divisor
    
    
    h_divisor = 0
    a_divisor = 0
    for j in range(len(past_game_coef)):
        h_home_stat_j = game_data['home_pass_att_middle'][h_games[j]] / game_data['home_pass_att'][h_games[j]] 
        h_away_stat_j = game_data['away_pass_att_middle'][h_games[j]] / game_data['away_pass_att'][h_games[j]] 
        a_home_stat_j = game_data['home_pass_att_middle'][a_games[j]] / game_data['home_pass_att'][a_games[j]]
        a_away_stat_j = game_data['away_pass_att_middle'][a_games[j]] / game_data['away_pass_att'][a_games[j]]
        
        if(game_data['home_team_code'][h_games[j]] == h_team):
            home_off_perc_pass_middle[i] += past_game_coef[j]*match_visit_stat_coef*h_home_stat_j
            home_def_perc_pass_middle[i] += past_game_coef[j]*match_visit_stat_coef*h_away_stat_j
            h_divisor += past_game_coef[j]*match_visit_stat_coef
        else:
            home_off_perc_pass_middle[i] += past_game_coef[j]*non_match_visit_stat_coef*h_away_stat_j
            home_def_perc_pass_middle[i] += past_game_coef[j]*non_match_visit_stat_coef*h_home_stat_j
            h_divisor += past_game_coef[j]*non_match_visit_stat_coef
            
        if(game_data['home_team_code'][a_games[j]] == a_team):
            away_off_perc_pass_middle[i] += past_game_coef[j]*non_match_visit_stat_coef*a_home_stat_j
            away_def_perc_pass_middle[i] += past_game_coef[j]*non_match_visit_stat_coef*a_away_stat_j
            a_divisor += past_game_coef[j]*non_match_visit_stat_coef
        else:
            away_off_perc_pass_middle[i] += past_game_coef[j]*match_visit_stat_coef*a_away_stat_j
            away_def_perc_pass_middle[i] += past_game_coef[j]*match_visit_stat_coef*a_home_stat_j
            a_divisor += past_game_coef[j]*match_visit_stat_coef
    
    home_off_perc_pass_middle[i] = home_off_perc_pass_middle[i] / h_divisor
    away_off_perc_pass_middle[i] = away_off_perc_pass_middle[i] / a_divisor
    home_def_perc_pass_middle[i] = home_def_perc_pass_middle[i] / h_divisor
    away_def_perc_pass_middle[i] = away_def_perc_pass_middle[i] / a_divisor
    
    
    h_divisor = 0
    a_divisor = 0
    for j in range(len(past_game_coef)):
        h_home_stat_j = game_data['home_pass_compl'][h_games[j]] / game_data['home_pass_att'][h_games[j]] 
        h_away_stat_j = game_data['away_pass_compl'][h_games[j]] / game_data['away_pass_att'][h_games[j]] 
        a_home_stat_j = game_data['home_pass_compl'][a_games[j]] / game_data['home_pass_att'][a_games[j]]
        a_away_stat_j = game_data['away_pass_compl'][a_games[j]] / game_data['away_pass_att'][a_games[j]]
        
        if(game_data['home_team_code'][h_games[j]] == h_team):
            home_off_pass_compl_rate[i] += past_game_coef[j]*match_visit_stat_coef*h_home_stat_j
            home_def_pass_compl_rate[i] += past_game_coef[j]*match_visit_stat_coef*h_away_stat_j
            h_divisor += past_game_coef[j]*match_visit_stat_coef
        else:
            home_off_pass_compl_rate[i] += past_game_coef[j]*non_match_visit_stat_coef*h_away_stat_j
            home_def_pass_compl_rate[i] += past_game_coef[j]*non_match_visit_stat_coef*h_home_stat_j
            h_divisor += past_game_coef[j]*non_match_visit_stat_coef
            
        if(game_data['home_team_code'][a_games[j]] == a_team):
            away_off_pass_compl_rate[i] += past_game_coef[j]*non_match_visit_stat_coef*a_home_stat_j
            away_def_pass_compl_rate[i] += past_game_coef[j]*non_match_visit_stat_coef*a_away_stat_j
            a_divisor += past_game_coef[j]*non_match_visit_stat_coef
        else:
            away_off_pass_compl_rate[i] += past_game_coef[j]*match_visit_stat_coef*a_away_stat_j
            away_def_pass_compl_rate[i] += past_game_coef[j]*match_visit_stat_coef*a_home_stat_j
            a_divisor += past_game_coef[j]*match_visit_stat_coef
    
    home_off_pass_compl_rate[i] = home_off_pass_compl_rate[i] / h_divisor
    away_off_pass_compl_rate[i] = away_off_pass_compl_rate[i] / a_divisor
    home_def_pass_compl_rate[i] = home_def_pass_compl_rate[i] / h_divisor
    away_def_pass_compl_rate[i] = away_def_pass_compl_rate[i] / a_divisor
    
    
    h_divisor = 0
    a_divisor = 0
    for j in range(len(past_game_coef)):
        h_home_stat_j = ((game_data['home_gross_pass_yds'][h_games[j]] + 20*game_data['home_pass_tds'][h_games[j]] 
                         - 45*game_data['home_ints_thrown'][h_games[j]] - game_data['home_sack_yds_taken'][h_games[j]]) 
                         / (game_data['home_pass_att'][h_games[j]] + game_data['home_sacks_taken'][h_games[j]]))
            
        h_away_stat_j = ((game_data['away_gross_pass_yds'][h_games[j]] + 20*game_data['away_pass_tds'][h_games[j]] 
                         - 45*game_data['away_ints_thrown'][h_games[j]] - game_data['away_sack_yds_taken'][h_games[j]]) 
                         / (game_data['away_pass_att'][h_games[j]] + game_data['away_sacks_taken'][h_games[j]]))
            
        a_home_stat_j = ((game_data['home_gross_pass_yds'][a_games[j]] + 20*game_data['home_pass_tds'][a_games[j]] 
                         - 45*game_data['home_ints_thrown'][a_games[j]] - game_data['home_sack_yds_taken'][a_games[j]]) 
                          / (game_data['home_pass_att'][a_games[j]] + game_data['home_sacks_taken'][a_games[j]]))
        
        a_away_stat_j = ((game_data['away_gross_pass_yds'][a_games[j]] + 20*game_data['away_pass_tds'][a_games[j]] 
                          - 45*game_data['away_ints_thrown'][a_games[j]] - game_data['away_sack_yds_taken'][a_games[j]]) 
                         / (game_data['away_pass_att'][a_games[j]] + game_data['away_sacks_taken'][a_games[j]]))
        
        if(game_data['home_team_code'][h_games[j]] == h_team):
            home_off_adj_net_yds_per_att[i] += past_game_coef[j]*match_visit_stat_coef*h_home_stat_j
            home_def_adj_net_yds_per_att[i] += past_game_coef[j]*match_visit_stat_coef*h_away_stat_j
            h_divisor += past_game_coef[j]*match_visit_stat_coef
        else:
            home_off_adj_net_yds_per_att[i] += past_game_coef[j]*non_match_visit_stat_coef*h_away_stat_j
            home_def_adj_net_yds_per_att[i] += past_game_coef[j]*non_match_visit_stat_coef*h_home_stat_j
            h_divisor += past_game_coef[j]*non_match_visit_stat_coef
            
        if(game_data['home_team_code'][a_games[j]] == a_team):
            away_off_adj_net_yds_per_att[i] += past_game_coef[j]*non_match_visit_stat_coef*a_home_stat_j
            away_def_adj_net_yds_per_att[i] += past_game_coef[j]*non_match_visit_stat_coef*a_away_stat_j
            a_divisor += past_game_coef[j]*non_match_visit_stat_coef
        else:
            away_off_adj_net_yds_per_att[i] += past_game_coef[j]*match_visit_stat_coef*a_away_stat_j
            away_def_adj_net_yds_per_att[i] += past_game_coef[j]*match_visit_stat_coef*a_home_stat_j
            a_divisor += past_game_coef[j]*match_visit_stat_coef
    
    home_off_adj_net_yds_per_att[i] = home_off_adj_net_yds_per_att[i] / h_divisor
    away_off_adj_net_yds_per_att[i] = away_off_adj_net_yds_per_att[i] / a_divisor
    home_def_adj_net_yds_per_att[i] = home_def_adj_net_yds_per_att[i] / h_divisor
    away_def_adj_net_yds_per_att[i] = away_def_adj_net_yds_per_att[i] / a_divisor
    
    
    fill_feature_1(home_off_perc_frthdown_att, away_off_perc_frthdown_att, home_def_perc_frthdown_att, 
                   away_def_perc_frthdown_att, 'fourth_down_att', 'fourth_downs')
    
    
    fill_feature_1(home_off_frthdown_conv, away_off_frthdown_conv, home_def_frthdown_conv, away_def_frthdown_conv, 
                 'fourth_down_suc', 'fourth_down_att')
    
    
    fill_feature_1(home_off_rze, away_off_rze, home_def_rze, away_def_rze, 'rz_tds', 'rz_trips')
    
    
    fill_feature_2(home_pat, away_pat, 'pat_m', 'pat_a')
    
    
    fill_feature_2(home_FG_39, away_FG_39, 'fgm_39', 'fga_39')
    
    
    fill_feature_2(home_FG_49, away_FG_49, 'fgm_40_49', 'fga_40_49')
    
    
    fill_feature_2(home_FG_50, away_FG_50, 'fgm_50', 'fga_50')
    
    
    fill_feature_2(home_yds_per_punt, away_yds_per_punt, 'punt_yds', 'punts')
    
    
    fill_feature_2(home_punts_in_20, away_punts_in_20, 'punts_inside_20', 'punts')
    
    
    fill_feature_1(home_off_2pt_conv, away_off_2pt_conv, home_def_2pt_conv, away_def_2pt_conv,
                   '2pt_conv_suc', '2pt_att')
    
    
    fill_feature_1(home_off_yds_per_punt_return, away_off_yds_per_punt_return, home_def_yds_per_punt_return, 
                   away_def_yds_per_punt_return, 'punt_return_yds', 'punt_returns')
    
    
    fill_feature_1(home_off_perc_kickoffs_returned, away_off_perc_kickoffs_returned, home_def_perc_kickoffs_returned, 
                   away_def_perc_kickoffs_returned, 'kickoff_returns', 'kickoffs_received')
    
    
    fill_feature_1(home_off_yds_per_kickoff_return, away_off_yds_per_kickoff_return, home_def_yds_per_kickoff_return, 
                   away_def_yds_per_kickoff_return, 'kickoff_return_yds', 'kickoff_returns')
    
    
    fill_feature_1(home_off_short_pass_compl_rate, away_off_short_pass_compl_rate, home_def_short_pass_compl_rate, 
                   away_def_short_pass_compl_rate, 'short_completions', 'short_pass_att')
    
    
    fill_feature_1(home_off_deep_pass_compl_rate, away_off_deep_pass_compl_rate, home_def_deep_pass_compl_rate, 
                   away_def_deep_pass_compl_rate, 'deep_completions', 'deep_pass_att')
    
    
    fill_feature_1(home_off_pass_compl_rate_middle, away_off_pass_compl_rate_middle, home_def_pass_compl_rate_middle, 
                   away_def_pass_compl_rate_middle, 'completions_middle', 'pass_att_middle')
    
    
    home_off_divisor = 0
    home_def_divisor = 0
    away_off_divisor = 0
    away_def_divisor = 0
    
    home_off_past_game_coef = copy.copy(past_game_coef)
    home_def_past_game_coef = copy.copy(past_game_coef)
    away_off_past_game_coef = copy.copy(past_game_coef)
    away_def_past_game_coef = copy.copy(past_game_coef)
    
    home_off_stats = np.zeros((len(past_game_coef)))
    away_off_stats = np.zeros((len(past_game_coef)))
    home_def_stats = np.zeros((len(past_game_coef)))
    away_def_stats = np.zeros((len(past_game_coef)))
    
    for j in range(len(past_game_coef)):
        
        weight_sum = 0
        atts = game_data['home_pat_a'][h_games[j]] + game_data['home_2pt_att'][h_games[j]]
        if(atts != 0):
            stat = game_data['home_2pt_att'][h_games[j]] / atts
            if(game_data['home_team_code'][h_games[j]] == h_team):
                home_off_stats[j] = stat
            else:
                home_def_stats[j] = stat
        else:
            if(game_data['home_team_code'][h_games[j]] == h_team):
                home_off_stats[j] = 0
                home_off_past_game_coef[j] = 0
                for n in range(len(past_game_coef)):
                    weight_sum += home_off_past_game_coef[n]
                    
                if(weight_sum != 0):
                    for m in range(len(past_game_coef)):
                        home_off_past_game_coef[m] = home_off_past_game_coef[m] * (1 / weight_sum)
            else:
                home_def_stats[j] = 0
                home_def_past_game_coef[j] = 0
                for n in range(len(past_game_coef)):
                    weight_sum += home_def_past_game_coef[n]
            
                if(weight_sum != 0):
                    for m in range(len(past_game_coef)):
                        home_def_past_game_coef[m] = home_def_past_game_coef[m] * (1 / weight_sum)
                        
        weight_sum = 0
        atts = game_data['away_pat_a'][h_games[j]] + game_data['away_2pt_att'][h_games[j]]
        if(atts != 0):
            stat =  game_data['away_2pt_att'][h_games[j]] / atts
            if(game_data['away_team_code'][h_games[j]] == h_team):
                home_off_stats[j] = stat
            else:
                home_def_stats[j] = stat
        else:
            if(game_data['away_team_code'][h_games[j]] == h_team):
                home_off_stats[j] = 0
                home_off_past_game_coef[j] = 0
                for n in range(len(past_game_coef)):
                    weight_sum += home_off_past_game_coef[n]
                    
                if(weight_sum != 0):
                    for m in range(len(past_game_coef)):
                        home_off_past_game_coef[m] = home_off_past_game_coef[m] * (1 / weight_sum)
            else:
                home_def_stats[j] = 0
                home_def_past_game_coef[j] = 0
                for n in range(len(past_game_coef)):
                    weight_sum += home_def_past_game_coef[n]
            
                if(weight_sum != 0):
                    for m in range(len(past_game_coef)):
                        home_def_past_game_coef[m] = home_def_past_game_coef[m] * (1 / weight_sum)
                
        weight_sum = 0
        atts = game_data['home_pat_a'][a_games[j]] + game_data['home_2pt_att'][a_games[j]]
        if(atts != 0):
            stat =  game_data['home_2pt_att'][a_games[j]] / atts
            if(game_data['home_team_code'][a_games[j]] == a_team):
                away_off_stats[j] = stat
            else:
                away_def_stats[j] = stat
        else:
            if(game_data['home_team_code'][a_games[j]] == a_team):
                away_off_stats[j] = 0
                away_off_past_game_coef[j] = 0
                for n in range(len(past_game_coef)):
                    weight_sum += away_off_past_game_coef[n]
                    
                if(weight_sum != 0):
                    for m in range(len(past_game_coef)):
                        away_off_past_game_coef[m] = away_off_past_game_coef[m] * (1 / weight_sum)
            else:
                away_def_stats[j] = 0
                away_def_past_game_coef[j] = 0
                for n in range(len(past_game_coef)):
                    weight_sum += away_def_past_game_coef[n]
            
                if(weight_sum != 0):
                    for m in range(len(past_game_coef)):
                        away_def_past_game_coef[m] = away_def_past_game_coef[m] * (1 / weight_sum)
            
        weight_sum = 0
        atts = game_data['away_pat_a'][a_games[j]] + game_data['away_2pt_att'][a_games[j]]
        if(atts != 0):
            stat =  game_data['away_2pt_att'][a_games[j]] / atts
            if(game_data['away_team_code'][a_games[j]] == a_team):
                away_off_stats[j] = stat
            else:
                away_def_stats[j] = stat
        else:
            if(game_data['away_team_code'][a_games[j]] == a_team):
                away_off_stats[j] = 0
                away_off_past_game_coef[j] = 0
                for n in range(len(past_game_coef)):
                    weight_sum += away_off_past_game_coef[n]
                    
                if(weight_sum != 0):
                    for m in range(len(past_game_coef)):
                        away_off_past_game_coef[m] = away_off_past_game_coef[m] * (1 / weight_sum)
            else:
                away_def_stats[j] = 0
                away_def_past_game_coef[j] = 0
                for n in range(len(past_game_coef)):
                    weight_sum += away_def_past_game_coef[n]
            
                if(weight_sum != 0):
                    for m in range(len(past_game_coef)):
                        away_def_past_game_coef[m] = away_def_past_game_coef[m] * (1 / weight_sum)
                        
    for j in range(len(past_game_coef)):
        if(game_data['home_team_code'][h_games[j]] == h_team):
            home_off_perc_2pt_att[i] += home_off_past_game_coef[j]*match_visit_stat_coef*home_off_stats[j]
            home_def_perc_2pt_att[i] += home_def_past_game_coef[j]*match_visit_stat_coef*home_def_stats[j]
            home_off_divisor += home_off_past_game_coef[j]*match_visit_stat_coef
            home_def_divisor += home_def_past_game_coef[j]*match_visit_stat_coef
        else:
            home_off_perc_2pt_att[i] += home_off_past_game_coef[j]*non_match_visit_stat_coef*home_off_stats[j]
            home_def_perc_2pt_att[i] += home_def_past_game_coef[j]*non_match_visit_stat_coef*home_def_stats[j]
            home_off_divisor += home_off_past_game_coef[j]*non_match_visit_stat_coef
            home_def_divisor += home_def_past_game_coef[j]*non_match_visit_stat_coef
            
        if(game_data['home_team_code'][a_games[j]] == a_team):
            away_off_perc_2pt_att[i] += away_off_past_game_coef[j]*non_match_visit_stat_coef*away_off_stats[j]
            away_def_perc_2pt_att[i] += away_def_past_game_coef[j]*non_match_visit_stat_coef*away_def_stats[j]
            away_off_divisor += away_off_past_game_coef[j]*non_match_visit_stat_coef
            away_def_divisor += away_def_past_game_coef[j]*non_match_visit_stat_coef
        else:
            away_off_perc_2pt_att[i] += away_off_past_game_coef[j]*match_visit_stat_coef*away_off_stats[j]
            away_def_perc_2pt_att[i] += away_def_past_game_coef[j]*match_visit_stat_coef*away_def_stats[j]
            away_off_divisor += away_off_past_game_coef[j]*match_visit_stat_coef
            away_def_divisor += away_def_past_game_coef[j]*match_visit_stat_coef
    
    if(home_off_divisor == 0):
        home_off_perc_2pt_att[i] = 0
    else:
        home_off_perc_2pt_att[i] = home_off_perc_2pt_att[i] / home_off_divisor
    if(away_off_divisor == 0):
        away_off_perc_2pt_att[i] = 0
    else:
        away_off_perc_2pt_att[i] = away_off_perc_2pt_att[i] / away_off_divisor
    if(home_def_divisor == 0):
        home_def_perc_2pt_att[i] = 0
    else:
        home_def_perc_2pt_att[i] = home_def_perc_2pt_att[i] / home_def_divisor
    if(away_def_divisor == 0):
        away_def_perc_2pt_att[i] = 0
    else:
        away_def_perc_2pt_att[i] = away_def_perc_2pt_att[i] / away_def_divisor
    
    
    prev_dates = previous_games['game_date'].to_numpy()
    month = game_date.month
    year =  game_date.year
    
    season_games = pd.DataFrame()
    h_season_home_games = pd.DataFrame()
    h_season_away_games = pd.DataFrame()
    a_season_home_games = pd.DataFrame()
    a_season_away_games = pd.DataFrame()
    
    try:
        if(month > 6):
            fst_game_date = prev_dates[prev_dates > datetime.strptime('7/1/'+str(year), '%m/%d/%Y')][0]
        else:
            fst_game_date = prev_dates[prev_dates > datetime.strptime('7/1/'+str(year-1), '%m/%d/%Y')][0]

        fst_game_i = game_data.index[game_data['game_date']==fst_game_date][0]


        if(fst_game_i != i):
            season_games = previous_games[fst_game_i:]
            h_season_home_games = season_games[season_games['home_team_code']==h_team]
            h_season_away_games = season_games[season_games['away_team_code']==h_team]
            a_season_home_games = season_games[season_games['home_team_code']==a_team]
            a_season_away_games = season_games[season_games['away_team_code']==a_team]
    except:
        pass
    
    h_season_games = pd.concat([h_season_home_games, h_season_away_games]).reset_index(drop=True)
    a_season_games = pd.concat([a_season_home_games, a_season_away_games]).reset_index(drop=True)
    
    h_margin_o_u = 0
    a_margin_o_u = 0
    h_margin_ats = 0
    a_margin_ats = 0
    h_record_margin = 0
    a_record_margin = 0
    h_pt_margin_o_u = 0
    a_pt_margin_o_u = 0
    h_pt_margin_ats = 0
    a_pt_margin_ats = 0
    h_pt_margin = 0
    a_pt_margin = 0
    
    if(len(h_season_games) > 0):
        for j in range(len(h_season_games)):
            if(h_season_games['home_pts'][j] + h_season_games['away_pts'][j] > h_season_games['vegas_o_u'][j]):
                h_margin_o_u += 1
            elif(h_season_games['home_pts'][j] + h_season_games['away_pts'][j] < h_season_games['vegas_o_u'][j]):
                h_margin_o_u -= 1

            h_pt_margin_o_u += (h_season_games['home_pts'][j] + h_season_games['away_pts'][j]) - h_season_games['vegas_o_u'][j]

            if(h_season_games['home_team_code'][j]==h_team):
                if(h_season_games['away_pts'][j] - h_season_games['home_pts'][j] < h_season_games['vegas_home_spread'][j]):
                    h_margin_ats += 1
                elif(h_season_games['away_pts'][j] - h_season_games['home_pts'][j] > h_season_games['vegas_home_spread'][j]):
                    h_margin_ats -= 1
                if(h_season_games['home_pts'][j] > h_season_games['away_pts'][j]):
                    h_record_margin += 1
                elif(h_season_games['home_pts'][j] < h_season_games['away_pts'][j]):
                    h_record_margin -= 1

                h_pt_margin_ats += h_season_games['vegas_home_spread'][j] - (h_season_games['away_pts'][j] 
                                                                             - h_season_games['home_pts'][j])

                h_pt_margin += h_season_games['home_pts'][j] - h_season_games['away_pts'][j]
            else:
                if(h_season_games['away_pts'][j] - h_season_games['home_pts'][j] < h_season_games['vegas_home_spread'][j]):
                    h_margin_ats -= 1
                elif(h_season_games['away_pts'][j] - h_season_games['home_pts'][j] > h_season_games['vegas_home_spread'][j]):
                    h_margin_ats += 1
                if(h_season_games['home_pts'][j] > h_season_games['away_pts'][j]):
                    h_record_margin -= 1
                elif(h_season_games['home_pts'][j] < h_season_games['away_pts'][j]):
                    h_record_margin += 1

                h_pt_margin_ats += -1*h_season_games['vegas_home_spread'][j] - (h_season_games['home_pts'][j] 
                                                                                - h_season_games['away_pts'][j])

                h_pt_margin += h_season_games['away_pts'][j] - h_season_games['home_pts'][j]
    if(len(a_season_games) > 0):
        for j in range(len(a_season_games)):
            if(a_season_games['home_pts'][j] + a_season_games['away_pts'][j] > a_season_games['vegas_o_u'][j]):
                a_margin_o_u += 1
            elif(a_season_games['home_pts'][j] + a_season_games['away_pts'][j] < a_season_games['vegas_o_u'][j]):
                a_margin_o_u -= 1

            a_pt_margin_o_u += (a_season_games['home_pts'][j] + a_season_games['away_pts'][j]) - a_season_games['vegas_o_u'][j]

            if(a_season_games['home_team_code'][j]==a_team):
                if(a_season_games['away_pts'][j] - a_season_games['home_pts'][j] < a_season_games['vegas_home_spread'][j]):
                    a_margin_ats += 1
                elif(a_season_games['away_pts'][j] - a_season_games['home_pts'][j] > a_season_games['vegas_home_spread'][j]):
                    a_margin_ats -= 1
                if(a_season_games['home_pts'][j] > a_season_games['away_pts'][j]):
                    a_record_margin += 1
                elif(a_season_games['home_pts'][j] < a_season_games['away_pts'][j]):
                    a_record_margin -= 1

                a_pt_margin_ats += a_season_games['vegas_home_spread'][j] - (a_season_games['away_pts'][j] 
                                                                             - a_season_games['home_pts'][j])

                a_pt_margin += a_season_games['home_pts'][j] - a_season_games['away_pts'][j]
            else:
                if(a_season_games['away_pts'][j] - a_season_games['home_pts'][j] < a_season_games['vegas_home_spread'][j]):
                    a_margin_ats -= 1
                elif(a_season_games['away_pts'][j] - a_season_games['home_pts'][j] > a_season_games['vegas_home_spread'][j]):
                    a_margin_ats += 1
                if(a_season_games['home_pts'][j] > a_season_games['away_pts'][j]):
                    a_record_margin -= 1
                elif(a_season_games['home_pts'][j] < a_season_games['away_pts'][j]):
                    a_record_margin += 1

                a_pt_margin_ats += -1*a_season_games['vegas_home_spread'][j] - (a_season_games['home_pts'][j] 
                                                                                - a_season_games['away_pts'][j])

                a_pt_margin += a_season_games['away_pts'][j] - a_season_games['home_pts'][j]
    
    
    home_margin_o_u[i] = h_margin_o_u
    away_margin_o_u[i] = a_margin_o_u
    home_margin_ats[i] = h_margin_ats
    away_margin_ats[i] = a_margin_ats
    home_record_margin[i] = h_record_margin
    away_record_margin[i] = a_record_margin
    home_pt_margin_o_u[i] = h_pt_margin_o_u
    away_pt_margin_o_u[i] = a_pt_margin_o_u
    home_pt_margin_ats[i] = h_pt_margin_ats
    away_pt_margin_ats[i] = a_pt_margin_ats
    home_pt_margin[i] = h_pt_margin
    away_pt_margin[i] = a_pt_margin
    

In [16]:
# create full data feature array
X = np.hstack((indoor_stadium, divisional, stadium, game_time, referee, temp, humidity, wind_speed, vegas_o_u, o_u_result, 
               vegas_home_spread, home_covered_spread, team_matchups_over, team_matchups_under, team_matchups_h_covered,
               team_matchups_a_covered, home_coach, away_coach, home_rest_days, away_rest_days, home_pos_time, 
               away_pos_time, home_off_pts_per_game, away_off_pts_per_game, home_def_pts_per_game, away_def_pts_per_game,
               home_off_pts_per_sec_half, away_off_pts_per_sec_half, home_def_pts_per_sec_half, away_def_pts_per_sec_half,
               home_off_pts_per_play, away_off_pts_per_play, home_def_pts_per_play, away_def_pts_per_play, 
               home_off_yds_per_play, away_off_yds_per_play, home_def_yds_per_play, away_def_yds_per_play, 
               home_off_time_per_play, away_off_time_per_play, home_def_time_per_play, away_def_time_per_play, 
               home_off_perc_rush, away_off_perc_rush, home_def_perc_rush, away_def_perc_rush, home_off_perc_fstdown_rush,
               away_off_perc_fstdown_rush, home_def_perc_fstdown_rush, away_def_perc_fstdown_rush, home_off_epr, away_off_epr,
               home_def_epr, away_def_epr, home_off_fumble_rate, away_off_fumble_rate, home_def_fumble_rate, 
               away_def_fumble_rate, home_off_int_rate, away_off_int_rate, home_def_int_rate, away_def_int_rate, 
               home_off_sack_rate, away_off_sack_rate, home_def_sack_rate, away_def_sack_rate, 
               home_turnover_margin, away_turnover_margin, home_lw_turnover_margin, away_lw_turnover_margin, 
               home_off_thrdown_conv, away_off_thrdown_conv, home_def_thrdown_conv, away_def_thrdown_conv, 
               home_off_perc_frthdown_att, away_off_perc_frthdown_att, home_def_perc_frthdown_att, away_def_perc_frthdown_att,
               home_off_frthdown_conv, away_off_frthdown_conv, home_def_frthdown_conv, away_def_frthdown_conv, home_off_rze,
               away_off_rze, home_def_rze, away_def_rze, home_pat, away_pat, home_FG_39, away_FG_39, home_FG_49, away_FG_49,
               home_FG_50, away_FG_50, home_yds_per_punt, away_yds_per_punt, home_punts_in_20, away_punts_in_20, 
               home_def_pts_100, away_def_pts_100, home_off_2pt_conv, away_off_2pt_conv, home_def_2pt_conv, away_def_2pt_conv,
               home_off_perc_2pt_att, away_off_perc_2pt_att, home_def_perc_2pt_att, away_def_perc_2pt_att, 
               home_off_yds_per_punt_return, away_off_yds_per_punt_return, home_def_yds_per_punt_return, 
               away_def_yds_per_punt_return, home_off_perc_kickoffs_returned, away_off_perc_kickoffs_returned, 
               home_def_perc_kickoffs_returned, away_def_perc_kickoffs_returned, home_off_yds_per_kickoff_return, 
               away_off_yds_per_kickoff_return, home_def_yds_per_kickoff_return, away_def_yds_per_kickoff_return,
               home_off_short_pass_compl_rate, away_off_short_pass_compl_rate, home_def_short_pass_compl_rate, 
               away_def_short_pass_compl_rate, home_off_deep_pass_compl_rate, away_off_deep_pass_compl_rate, 
               home_def_deep_pass_compl_rate, away_def_deep_pass_compl_rate, home_off_pass_compl_rate_middle, 
               away_off_pass_compl_rate_middle, home_def_pass_compl_rate_middle, away_def_pass_compl_rate_middle,
               home_off_avg_starting_field_pos, away_off_avg_starting_field_pos, home_def_avg_starting_field_pos,
               away_def_avg_starting_field_pos, home_off_pen_yds, away_off_pen_yds, home_def_pen_yds, away_def_pen_yds,
               home_opp_off_pen_yds, away_opp_off_pen_yds, home_opp_def_pen_yds, away_opp_def_pen_yds, home_off_rush_ypc,
               away_off_rush_ypc, home_def_rush_ypc, away_def_rush_ypc, home_off_perc_rush_ends, away_off_perc_rush_ends,
               home_def_perc_rush_ends, away_def_perc_rush_ends, home_off_early_down_rush_suc, away_off_early_down_rush_suc,
               home_def_early_down_rush_suc, away_def_early_down_rush_suc, home_off_early_down_pass_suc, 
               away_off_early_down_pass_suc, home_def_early_down_pass_suc, away_def_early_down_pass_suc, 
               home_off_perc_pass_middle, away_off_perc_pass_middle, home_def_perc_pass_middle, away_def_perc_pass_middle,
               home_off_pass_compl_rate, away_off_pass_compl_rate, home_def_pass_compl_rate, away_def_pass_compl_rate,
               home_off_adj_net_yds_per_att, away_off_adj_net_yds_per_att, home_def_adj_net_yds_per_att, 
               away_def_adj_net_yds_per_att, home_margin_o_u, away_margin_o_u, home_margin_ats, away_margin_ats, 
               home_record_margin, away_record_margin, home_pt_margin_o_u, away_pt_margin_o_u, home_pt_margin_ats,
               away_pt_margin_ats, home_pt_margin, away_pt_margin))

X.shape

(4078, 184)

In [17]:
#createlistofcolumnnamesforallfeatures
col_names=['indoor_stadium','divisional','stadium','game_time','referee','temp','humidity','wind_speed','vegas_o_u',
           'o_u_result','vegas_home_spread','home_covered_spread','team_matchups_over','team_matchups_under',
           'team_matchups_h_covered','team_matchups_a_covered','home_coach','away_coach','home_rest_days',
           'away_rest_days','home_pos_time','away_pos_time','home_off_pts_per_game','away_off_pts_per_game',
           'home_def_pts_per_game','away_def_pts_per_game','home_off_pts_per_sec_half','away_off_pts_per_sec_half',
           'home_def_pts_per_sec_half','away_def_pts_per_sec_half','home_off_pts_per_play','away_off_pts_per_play',
           'home_def_pts_per_play','away_def_pts_per_play','home_off_yds_per_play','away_off_yds_per_play',
           'home_def_yds_per_play','away_def_yds_per_play','home_off_time_per_play','away_off_time_per_play',
           'home_def_time_per_play','away_def_time_per_play','home_off_perc_rush','away_off_perc_rush','home_def_perc_rush',
           'away_def_perc_rush','home_off_perc_fstdown_rush','away_off_perc_fstdown_rush','home_def_perc_fstdown_rush',
           'away_def_perc_fstdown_rush','home_off_epr','away_off_epr','home_def_epr','away_def_epr','home_off_fumble_rate',
           'away_off_fumble_rate','home_def_fumble_rate','away_def_fumble_rate','home_off_int_rate','away_off_int_rate',
           'home_def_int_rate','away_def_int_rate','home_off_sack_rate','away_off_sack_rate','home_def_sack_rate',
           'away_def_sack_rate','home_turnover_margin','away_turnover_margin','home_lw_turnover_margin',
           'away_lw_turnover_margin','home_off_thrdown_conv','away_off_thrdown_conv','home_def_thrdown_conv',
           'away_def_thrdown_conv','home_off_perc_frthdown_att','away_off_perc_frthdown_att','home_def_perc_frthdown_att',
           'away_def_perc_frthdown_att','home_off_frthdown_conv','away_off_frthdown_conv','home_def_frthdown_conv',
           'away_def_frthdown_conv','home_off_rze','away_off_rze','home_def_rze','away_def_rze','home_pat','away_pat',
           'home_FG_39','away_FG_39','home_FG_49','away_FG_49','home_FG_50','away_FG_50','home_yds_per_punt',
           'away_yds_per_punt','home_punts_in_20','away_punts_in_20','home_def_pts_100','away_def_pts_100','home_off_2pt_conv',
           'away_off_2pt_conv','home_def_2pt_conv','away_def_2pt_conv','home_off_perc_2pt_att','away_off_perc_2pt_att',
           'home_def_perc_2pt_att','away_def_perc_2pt_att','home_off_yds_per_punt_return','away_off_yds_per_punt_return',
           'home_def_yds_per_punt_return','away_def_yds_per_punt_return','home_off_perc_kickoffs_returned',
           'away_off_perc_kickoffs_returned','home_def_perc_kickoffs_returned','away_def_perc_kickoffs_returned',
           'home_off_yds_per_kickoff_return','away_off_yds_per_kickoff_return','home_def_yds_per_kickoff_return',
           'away_def_yds_per_kickoff_return','home_off_short_pass_compl_rate','away_off_short_pass_compl_rate',
           'home_def_short_pass_compl_rate','away_def_short_pass_compl_rate','home_off_deep_pass_compl_rate',
           'away_off_deep_pass_compl_rate','home_def_deep_pass_compl_rate','away_def_deep_pass_compl_rate',
           'home_off_pass_compl_rate_middle','away_off_pass_compl_rate_middle','home_def_pass_compl_rate_middle',
           'away_def_pass_compl_rate_middle','home_off_avg_starting_field_pos','away_off_avg_starting_field_pos',
           'home_def_avg_starting_field_pos','away_def_avg_starting_field_pos','home_off_pen_yds','away_off_pen_yds',
           'home_def_pen_yds','away_def_pen_yds','home_opp_off_pen_yds','away_opp_off_pen_yds','home_opp_def_pen_yds',
           'away_opp_def_pen_yds','home_off_rush_ypc','away_off_rush_ypc','home_def_rush_ypc','away_def_rush_ypc',
           'home_off_perc_rush_ends','away_off_perc_rush_ends','home_def_perc_rush_ends','away_def_perc_rush_ends',
           'home_off_early_down_rush_suc','away_off_early_down_rush_suc','home_def_early_down_rush_suc',
           'away_def_early_down_rush_suc','home_off_early_down_pass_suc','away_off_early_down_pass_suc',
           'home_def_early_down_pass_suc','away_def_early_down_pass_suc','home_off_perc_pass_middle',
           'away_off_perc_pass_middle','home_def_perc_pass_middle','away_def_perc_pass_middle','home_off_pass_compl_rate',
           'away_off_pass_compl_rate','home_def_pass_compl_rate','away_def_pass_compl_rate','home_off_adj_net_yds_per_att',
           'away_off_adj_net_yds_per_att','home_def_adj_net_yds_per_att','away_def_adj_net_yds_per_att','home_margin_o_u',
           'away_margin_o_u','home_margin_ats','away_margin_ats','home_record_margin','away_record_margin','home_pt_margin_o_u',
           'away_pt_margin_o_u','home_pt_margin_ats','away_pt_margin_ats','home_pt_margin','away_pt_margin']

In [18]:
# create final game data DataFrame
game_data_final = pd.DataFrame(X, columns = col_names)
# export as csv file
game_data_final.to_csv(r'C:\Users\Warren\OneDrive\Documents\NFL Bets\Model\nfl-transformed-game-data.csv', 
                       index=False, header=True)